# wikipedia 데이터 탐색
- `wikipedia_documents.json` 파일에 대한 탐색 내용입니다
- 살펴본 내용은 다음과 같습니다.

1. 데이터 중복 확인
    - document id에 중복 존재하는가? -> 중복 없음
    - text에 중복 존재하는가? -> 중복 있음 (7677개 text)
2. wiki 텍스트 단위 구성
    - 어떤 단위로 wiki 텍스트 구성된걸까? -> 제목을 기준으로 나뉜 듯 함
3. train/validation의 context와 wikipedia text의 관계
    - train/valiation 데이터셋은 모두 wikipedia에서 나온 passage들인가? -> 맞음
        - 따라서 텍스트에 대한 전처리 로직 만들어서, wikipedia, mrc(train, validation, test) text 데이터에 일괄적으로 전처리 수행하면 될 것으로 보임
4. wikipedia 문서 기준 전처리 할 내용 탐색
    1. 개행 (`\n`)
    2. 마크다운 문법 (`*`를 이용한 목록)
    3. 수식
    4. 미흡한 파싱 (`]]`)
    5. 미흡한 파싱 (문서 링크)
    6. 띄어쓰기 (붙여씀 / 2번 이상 띄어씀)

In [1]:
from datasets import load_from_disk
import pandas as pd
import json
from pprint import pprint

# # 각 cell에서 truncation 없이 모든 내용 출력
# pd.set_option('display.max_colwidth', None)

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# wikipedia 데이터 불러오기
with open("../data/wikipedia_documents.json", "r") as f:
    wiki = json.load(f)

wiki_df = pd.DataFrame(wiki)
wiki_df = wiki_df.transpose()
wiki_df = wiki_df[['title','text','document_id']]  # 필요한 column만 남기기
wiki_df.head()

,title,text,document_id
0,나라 목록,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",0
1,나라 목록,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,1
2,백남준,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,2
3,아오조라 문고,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",3
4,아오조라 문고,저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,4


In [3]:
# train, validation 데이터 불러오기 (wikipedia 데이터와의 일치 확인 목적)

# dataset을 pandas dataframe으로 변환하는 함수
def dataset_to_df(dataset_name):
    data_list = []

    # 필요한 항목 추출
    for item in dataset_name:
        data_list.append({
            'title': item['title'],
            'context': item['context'],
            'question': item['question'],
            'id': item['id'],
            'answer_start': item['answers']['answer_start'][0],
            'answer_text': item['answers']['text'][0],
            'document_id': item['document_id'],
        })

    # pandas dataframe으로 변환
    result_df = pd.DataFrame(data_list)
    return result_df


dataset = load_from_disk("../data/train_dataset/")
train_data, valid_data = dataset['train'], dataset['validation']
train_df = dataset_to_df(train_data)
valid_df = dataset_to_df(valid_data)

### 1.데이터 중복 확인
- document id에 중복 존재하는가? -> 중복 없음
- text에 중복 존재하는가? -> 중복 있음 (7677개 text)
    - 이유 1. 같은 문서(동일한 title)가 여러 번 수집 (7586/7677건)
    - 이유 2. 동일한 문장이 여러 문서에 사용된 경우 (91/7677건)

In [4]:
# Q. document_id에 중복 존재하는가? -> 중복 없음
# A. 중복 없음. 다음 두 숫자 일치하므로.
print(f"전체 문서 개수: {wiki_df.shape[0]}")
print(f"unique한 document id 개수: {len(wiki_df.document_id.unique())}")

전체 문서 개수: 60613
unique한 document id 개수: 60613


In [5]:
# Q. text에 중복 존재하는가?
# A. 중복 있음. 7677개 text에서 서로 중복 확인됨

# Q. 그럼 왜 중복 발생하나?
# A-1. 같은 문서(동일한 title)가 여러 번 수집된 경우 (대부분의 경우: 7586/7677건)
# A-2. 동일한 문장이 여러 문서에 사용된 경우 (91/7677건)

# 중복된 데이터 찾기 (text를 기준으로)
duplicates = wiki_df[wiki_df.duplicated(subset=['text'], keep=False)]

# title이 동일한 경우의 중복 건수
same_title_duplicates = duplicates[duplicates.duplicated(subset=['text', 'title'], keep=False)]

# title이 다른 경우의 중복 건수
different_title_duplicates = duplicates[~duplicates.duplicated(subset=['text', 'title'], keep=False)]

# unique한 text
wiki_df_unique = wiki_df.drop_duplicates(subset=['text'])

# 결과 출력
print(f"전체 중복 행 수: {len(duplicates)}")
print(f"Title이 동일한 중복 건수: {len(same_title_duplicates)}")
print(f"Title이 다른 중복 건수: {len(different_title_duplicates)}")
print("-"*50)
print(f"전체 wiki 데이터 수: {len(wiki_df)}")
print(f"text 중복 제거한 wiki 데이터 수: {len(wiki_df_unique)}")

전체 중복 행 수: 7677
Title이 동일한 중복 건수: 7586
Title이 다른 중복 건수: 91
--------------------------------------------------
전체 wiki 데이터 수: 60613
text 중복 제거한 wiki 데이터 수: 56737


In [ ]:
# 같은 문서가 여러 번 수집되어 중복 발생한 예시
wiki_df[wiki_df['title']=='로봇 배제 표준']

,title,text,document_id
3859,로봇 배제 표준,"""Robots.txt""는 이 문서를 가리킵니다. 위키백과의 Robots.txt의 파...",3859
3860,로봇 배제 표준,"만약 모든 로봇에게 문서 접근을 허락하려면, robots.txt에 다음과 같이 입력...",3860
8413,로봇 배제 표준,"""Robots.txt""는 이 문서를 가리킵니다. 위키백과의 Robots.txt의 파...",8413
8414,로봇 배제 표준,"만약 모든 로봇에게 문서 접근을 허락하려면, robots.txt에 다음과 같이 입력...",8414


In [ ]:
wiki_df[wiki_df['title']=='아미타불']

,title,text,document_id
3743,아미타불,"아미타불(阿彌陀佛), 무량광불(無量光佛) 또는 무량수불(無量壽佛)은 서방 극락세계(...",3743
3744,아미타불,"""아미타불""이라는 낱말은 무량광(無量光: 헤아릴 수 없는 광명을 가진 것)의 뜻인 ...",3744
3745,아미타불,"과거에, 아미타불이 되기 이전인 법장보살 때에 처음 세운 원으로서, 《불설무량수경》...",3745
8297,아미타불,"아미타불(阿彌陀佛), 무량광불(無量光佛) 또는 무량수불(無量壽佛)은 서방 극락세계(...",8297
8298,아미타불,"""아미타불""이라는 낱말은 무량광(無量光: 헤아릴 수 없는 광명을 가진 것)의 뜻인 ...",8298
8299,아미타불,"과거에, 아미타불이 되기 이전인 법장보살 때에 처음 세운 원으로서, 《불설무량수경》...",8299


In [ ]:
# 동일한 문장이 여러 문서에 사용도니 경우 예시
wiki_df[wiki_df.text == "김정은의 집권 이후 조선민주주의인민공화국은 예산제에서 전면적으로 독립채산제로 설정을 하였으며 특히 평양 집적회로 공장과 평양 집적회로 시험 공장과 리철호 가 사업하는 기계 공장에도 전면적으로 독립채산제를 갖추어 나가기 시작하였다.\n\n2014년에는 기업이 가격을 제정할 수 있는 상품의 범위를 확대한 것으로 여기에는 대학에서 첨단기술제품을 수요자와 계약에 따라 생산하여 내화 및 외화로 판매하는 제품의 가격과 주문계약으로 생산한 제품 가운데 외화로 원가를 보상하여야 할 제품의 외화가격 등이 포함되었다.\n\n특히 반도체나 특히 소프트웨어등을 개발하여 판매한 수익을 자체로 예산에 포함시켜서 수익사업에 나설 수 있도록 전면적으로 허용을 하였으며 특히 국정가격의 판매를 전면적으로 없애고 자율적으로 시장에 의하여 조절할 수 있도록 허용을 하였다."]

,title,text,document_id
3669,조선련봉총회사,김정은의 집권 이후 조선민주주의인민공화국은 예산제에서 전면적으로 독립채산제로 설정을...,3669
8223,조선련봉총회사,김정은의 집권 이후 조선민주주의인민공화국은 예산제에서 전면적으로 독립채산제로 설정을...,8223
56098,리철호가사업하는기계공장,김정은의 집권 이후 조선민주주의인민공화국은 예산제에서 전면적으로 독립채산제로 설정을...,56098
56106,평양집적회로시험공장,김정은의 집권 이후 조선민주주의인민공화국은 예산제에서 전면적으로 독립채산제로 설정을...,56106
58404,111호마스크제작소,김정은의 집권 이후 조선민주주의인민공화국은 예산제에서 전면적으로 독립채산제로 설정을...,58404
58793,조선백송회사,김정은의 집권 이후 조선민주주의인민공화국은 예산제에서 전면적으로 독립채산제로 설정을...,58793


In [ ]:
wiki_df[wiki_df.text == '삼마지(,\n,\n,\n,\n,\n,\n,\n,\n)는\n설일체유부의 5위 75법에서 심소법(心所法: 46가지) 중\n대지법(大地法: 10가지) 가운데 하나이며,\n유식유가행파와 법상종의 5위 100법에서 심소법(心所法: 51가지) 중\n별경심소(別境心所: 5가지) 가운데 하나이다.pp=69-81loc="三摩地". 2012년 9월 22일에 확인\n삼마지(三摩地)는 심일경성(心一境性)의 마음작용이다. 즉 마음(6식 또는 8식, 즉 심왕, 즉 심법)을 평등(平等)하게 유지하여 즉 혼침(惛沈)과 도거(掉舉)를 멀리 떠난 상태에서 그것이 하나의 대상에 전념(專念, 專住)하게 하는 마음작용이다. 달리 말하면, 마음(6식 또는 8식, 즉 심왕, 즉 심법)을 한 곳에 모아 산란하지 않게 하는 마음작용이다. 마치 뱀이 죽통(竹筒)에 들어가면 바로 펴지듯이 마음(6식 또는 8식, 즉 심왕, 즉 심법)이 삼마지에 들면 산란되지 않고 한결같게 된다. 문자 그대로 번역하여 () 삼마제(三摩提) 삼마제(三摩帝)라 하기도 하며, 의역하여 선정(禪定)loc="禪定(선정)". 2013년 3월 2일에 확인|ps=]]() 등지(等持) 정정(正定) 정의(定意) 조직정(調直定) 정심행처(正心行處)라 하기도 한다.pp=69-81세친 지음, 현장 한역, 권오민 번역|p=163 / 1397loc="三摩地". 2012년 9월 22일에 확인loc="三摩地(삼마지)". 2012년 9월 22일에 확인 (참고: 삼매())']

,title,text,document_id
32724,삼마지,"삼마지(,\n,\n,\n,\n,\n,\n,\n,\n)는\n설일체유부의 5위 75법에...",32724
33936,불교 용어 목록 (삼),"삼마지(,\n,\n,\n,\n,\n,\n,\n,\n)는\n설일체유부의 5위 75법에...",33936


### 2.wiki 텍스트 단위 구성
- 어떤 단위로 wiki 텍스트 구성된걸까? -> 제목을 기준으로 나뉜 듯 함

In [10]:
# Q. 어떤 단위로 wiki 텍스트 구성된걸까?
# A. 중제목/소제목 등 제목을 기준으로 나눈 것으로 보임. 위키피디아 페이지에서 해당 문서 제목 검색해서 직접 대조해봄

# 예시 1: '양태 (문법)' 문서 (https://ko.wikipedia.org/wiki/%EC%96%91%ED%83%9C_(%EB%AC%B8%EB%B2%95)#cite_note-FOOTNOTEBybee1994176-179-35)
wiki_df[wiki_df['title']=='양태 (문법)']

,title,text,document_id
59541,양태 (문법),양태에 대한 가장 유명한 정의는 언어학자 라이언스(John Lyons)의 ‘문장이 ...,59541
59542,양태 (문법),영국의 언어학자 프랭크 팔머(Frank Robert Palmer)는 양태를 명제 양...,59542
59543,양태 (문법),증거성(evidentiality) 또는 증거 양태(evidential modalit...,59543


In [11]:
# 예시 2: '당나라의 이슬람교' 문서 (https://ko.wikipedia.org/wiki/%EB%8B%B9%EB%82%98%EB%9D%BC%EC%9D%98_%EC%9D%B4%EC%8A%AC%EB%9E%8C%EA%B5%90)
wiki_df[wiki_df['title']=='당나라의 이슬람교']

,title,text,document_id
59599,당나라의 이슬람교,"중국 무슬림의 전승에 따르면, 이슬람은 성사 무함마드가 사망한 지 20년도 채 되지...",59599
59600,당나라의 이슬람교,"하마다 하그라스(Hamada Hagras)는 ""이슬람은 당 고종 2년 여름인 서기 ...",59600
59601,당나라의 이슬람교,초기 중국 이슬람 건축물은 장안의 대청진사(大清真寺)로 742년(내부의 석판에 새겨...,59601
59602,당나라의 이슬람교,당대 이슬람은 무역과 상업에 관심이 많았고 이슬람의 확산에는 전혀 관심이 없던 아랍...,59602
59603,당나라의 이슬람교,"그러나 당대에는 양주 대학살(760년), 광주 대학살 등 무슬림에 대한 두 차례의 ...",59603


### 3.train/validation의 context와 wikipedia text의 관계
- train/valiation 데이터셋은 모두 wikipedia에서 나온 passage들인가? -> 맞음
    - 따라서 텍스트에 대한 전처리 로직 만들어서, wikipedia, mrc(train, validation, test) text 데이터에 일괄적으로 전처리 수행하면 될 것으로 보임

In [12]:
# Q. train/valiation 데이터셋은 모두 wikipedia에서 나온 passage들인가?
# A. 맞음

# wiki_texts를 hash set으로 변환
wiki_text_hashes = set(hash(text) for text in wiki_df['text'])  

# 각 context의 hash를 구하고, wiki_text_hashes에 있는지 확인
train_df['is_wikipedia'] = train_df['context'].apply(lambda context: hash(context) in wiki_text_hashes) 
train_df[~train_df['is_wikipedia']] # Wikipedia에 없는 passage만 필터링 (is_wikipedia가 False인 행들)

,title,context,question,id,answer_start,answer_text,document_id,is_wikipedia


In [13]:
valid_df['is_wikipedia'] = valid_df['context'].apply(lambda context: hash(context) in wiki_text_hashes) 
valid_df[~valid_df['is_wikipedia']]

,title,context,question,id,answer_start,answer_text,document_id,is_wikipedia


### 4.wikipedia 문서 기준 전처리 할 내용 탐색
- 랜덤하게 문서들 샘플링해서, 눈으로 훑으며 전처리할 내용 탐색함 (정량적 수치 근거는 없음)
1. 개행 (`\n`)
    - 예시: `개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n`
2. 마크다운 문법 (`*`를 이용한 목록)
    - 예시: `위 기준에 논거하여 이 목록은 다음 206개 국가를 포함하고 있다.\n* 일반 국제 승인을 받은 195개 자주 국가.\n** 유엔 가입 국가 193개\n바티칸 시국\n`
3. 수식
    - 예시: `:: x×(a y + b z) = a x × y + b x × z and (a y + b z) × x = a y × x + b z × x`
4. 미흡한 파싱 (`]]`)
    - 예시: `앞줄 왼쪽부터 퍼시벌 로웰, 홍영식, 민영익, 서광범, 중국인 통역 우리탕.]]\n`
5. 미흡한 파싱 (문서 링크)
    - 예시: `디오판토스의 산술 (책)|label=산술|en|Arithmetica(3세기) 등이 있다.`
6. 띄어쓰기 (붙여씀 / 2번 이상 띄어씀)
    - 예시: `홈팀이 흰색,원정팀이 유색 유니폼을 입은   1987년  본인(정주영)의 고향인 강원도를 연고지로 설정했다가  1990년`

In [14]:
# 1. 개행 (`\n`)
pprint(wiki_df[wiki_df.document_id == 0].text.iloc[0])

('이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n'
 '\n'
 '이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n'
 '\n'
 '# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 '
 '나라를 나열하고 있다.\n'
 '# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 '
 '나라를 나열하고 있다.\n'
 '\n'
 '두 목록은 모두 가나다 순이다.\n'
 '\n'
 '일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 '
 '과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 '
 '정보는 "국가" 문서에서 설명하고 있다.')


In [15]:
# 2. 마크다운 문법 (`*`를 이용한 목록)
pprint(wiki_df[wiki_df.document_id == 1].text.iloc[0])

('이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정에 따르면, 국가는 다음의 조건을 만족해야 한다.\n'
 '* (a) 영속적인 국민\n'
 '* (b) 일정한 영토\n'
 '* (c) 정부\n'
 '* (d) 타국과의 관계 참여 자격.\n'
 '특히, 마지막 조건은 국제 공동체의 참여 용인을 내포하고 있기 때문에, 다른 나라의 승인이 매우 중요한 역할을 할 수 있다.  이 목록에 '
 '포함된 모든 국가는 보통 이 기준을 만족하는 것으로 보이는 자주적이고 독립적인 국가이다. 하지만 몬테비데오 협약 기준을 만족하는지의 '
 '여부는 많은 국가가 논쟁이 되고 있는 실정이다. 또한, 몬테비데오 협약 기준만이 국가 지위의 충분한 자격이든 아니든, 국제법의 견해 '
 '차이는 존재할 수 있다. 이 물음에 대한 다른 이론에 대한 고리는 아래에서 볼 수 있다.\n'
 '\n'
 '위 기준에 논거하여 이 목록은 다음 206개 국가를 포함하고 있다.\n'
 '* 일반 국제 승인을 받은 195개 자주 국가.\n'
 '** 유엔 가입 국가 193개\n'
 '** 성좌의 명칭으로 유엔에서 국제 승인을 받은 국가: 바티칸 시국\n'
 '** 팔레스타인지역에 위치하며 유엔 영구 옵서버 국가: 팔레스타인\n'
 '* 유엔의 가입국이 아니며, 일반 국제 승인을 받지 않은 11개 자주 국가.\n'
 '** 유엔 회원국으로부터 승인을 받은 8개 국가: 남오세티야, 니우에, 북키프로스 튀르크 공화국, 사하라 아랍 민주 공화국, 압하지야, '
 '중화민국, 코소보, 쿡 제도,\n'
 '** 유엔 비회원국으로부터 승인을 받은 2개 국가: 아르차흐 공화국, 트란스니스트리아\n'
 '** 어떤 나라에서도 승인 받지 않은 국가: 소말릴란드\n'
 '\n'
 '위 목록에 포함되지 않은 다음 국가는 몬테비데오 협약의 모든 조건을 만족하지 못하거나, 자주적이고 독립적임을 주장하지 않는 국가이다.\n'
 '* 남극 대륙 전체는 정부와 영속 인구가 없다. 7개 국가가 남극 

In [16]:
# 3. 수식
pprint(wiki_df[wiki_df.document_id == 32].text.iloc[0])

('7차원 벡터 공간의 벡터곱도 사원수의 방법을 팔원수에 적용하여 얻어질 수 있다.\n'
 '\n'
 '7차원 공간의 벡터곱은 다음과 같은 성질을 3차원 공간의 벡터곱과 공유한다.\n'
 '\n'
 '* 다음과 같은 의미에서 겹선형(bilinear)이다.\n'
 ':: x×(a y + b z) = a x × y + b x × z and (a y + b z) × x = a y × x + b z × '
 'x\n'
 '* 반가환성 (anti-commutative)\n'
 ':: x×y + y×x = 0\n'
 '* x와 y 모두에 수직\n'
 ':: x·(x×y) = y·(x×y) = 0\n'
 '* 야코비 항등식이 성립한다.\n'
 '::x×(y×z) + y×(z×x) + z×(x×y) = 0\n'
 '* ||x×y|| = ||x||||y||-(x·y)')


In [17]:
# 4. 미흡한 파싱 (`]]`)
pprint(wiki_df[wiki_df.document_id == 608].text.iloc[0])

('앞줄 왼쪽부터 퍼시벌 로웰, 홍영식, 민영익, 서광범, 중국인 통역 우리탕.]]\n'
 '보빙사(報聘使)는 1883년 조선에서 최초로 미국 등 서방 세계에 파견된 외교 사절단이다.\n'
 '\n'
 '1882년 조미 수호 통상 조약의 체결로 1883년 주한(駐韓) 공사 루시어스 푸트(Foote, L. H.)가 조선에 부임하였다. 이에 '
 '고종은 임오군란 이후 비대해진 청나라의 세력을 견제한다는 뜻에서 1883년 정사(正使)에 민영익(閔泳翊), 부사(副使)에 '
 '홍영식(洪英植), 서기관은 서광범(徐光範), 수행원은 변수(邊樹, 邊燧)·유길준(兪吉濬) 등 개화파 인사들을 대동시킨 친선 사절단을 서방 '
 '세계에 파견하였다. 사절단은 퍼시벌 로웰(Percival Lawrence Lowell)과 통역관 미야오카 츠네지로(宮岡恒次郎)의 인도 '
 '하에, 태평양을 건너 샌프란시스코에 도착하고 미대륙을 횡단한 다음 워싱턴을 거쳐 뉴욕에서 미국 대통령 체스터 A. 아서(Arthur, '
 'C. A.)와 2차례 회동하고 국서를 전하고 양국간의 우호와 교역에 관하여 논의하였다. 대통령과의 회동에서 보여준 조선식 전통 예법인 '
 '절은 소소한 충격을 주기도 했다. \n'
 '\n'
 '이어 세계박람회·시범농장·방직공장·의약제조회사·해군연병장·병원·전기회사·철도회사·소방서·육군사관학교 등 공공기관을 시찰하였다.\n'
 '특히, 워싱턴에서 내무성 교육국국장 이튼(Eaton, J.)을 방문하여 미국의 교육제도에 대하여 소개받았다. 뒤에 교육국사(敎育局史)와 '
 '연보를 기증받았다.\n'
 '그밖에 우편제도·전기시설·농업기술에 관심을 보였는데, 뒤에 우정국 설치, 경복궁의 전기설비, '
 '육영공원(育英公院)·농무목축시험장(農務牧畜試驗場) 등의 실현 계기가 되었다.\n'
 '\n'
 '이러한 인연으로 뒤에 주한미국공사 푸트를 통하여 육영공원 교사선발을 국장 이튼에게 의뢰, 뉴욕의 유니언 신학교(Union '
 'Theological Seminary)의 신학생 헐버트(

In [18]:
# 5. 미흡한 파싱 (문서 링크)
pprint(wiki_df[wiki_df.document_id == 4585].text.iloc[0])

('고대의 대수학에 대한 주요 저서로는 에우클레이데스의 원론(기원전 3세기)이나 구장산술(3세기), 디오판토스의 산술 '
 '(책)|label=산술|en|Arithmetica(3세기) 등이 있다. 9세기에 페르시아의 수학자 콰리즈미는 《약분·소거 '
 '계산론|en|The Compendious Book on Calculation by Completion and '
 'Balancing》(820년)를 통해 대수학을 하나의 독립적인 분야로 정립했다. 이 책은 1145년 체스터의 로버트|en|Robert '
 'of Chester가 《알게브라와 알무카발라의 서(書)》(Liber algebrae et almucabala)란 제목으로 라틴어로 번역한 '
 '뒤 다섯 세기에 걸쳐서 유럽의 대학에서 사용되었다. 여기서 "알게브라"(algebra)와 "알무카발라"(almucabala)는 해당하는 '
 '아랍어 단어를 음역한 것이다. 또한 콰리즈미의 저서인 "인도 수의 계산법"이 라틴어로 번역되면서 2차 방정식, 사칙연산, 십진법, 0 '
 '등의 개념이 소개되었다.\n'
 '\n'
 '19세기 이후에는 에바리스트 갈루아가 대수 방정식을 연구하기 위해서 군이라는 대수적 구조를 도입하였고, 조지 불은 논리학을 연구하기 '
 '위해서 불 대수라는 대수적 구조를 정의하였다. 이후 현대 수학에서는 다비트 힐베르트의 공리 주의나 니콜라 부르바키 스타일에서 찾아볼 수 '
 '있듯이, 고전적인 대수학에서 상당히 거리가 추상화되어 있으며, 방정식의 해법은 "방정식론"(대수방정식론)이라는 대수학의 일부분에 '
 '불과하다.')


In [19]:
# 6. 띄어쓰기 (붙여씀 / 2번 이상 띄어씀)
pprint(wiki_df[wiki_df.document_id == 39].text.iloc[0])

('1915년 강원도 통천군 답전면 아산리 (현 조선민주주의인민공화국 강원도 통천군 로상리)에서 아버지 정봉식과 어머니 한성실 사이에서 6남 '
 '2녀 중 장남으로 태어났다.  아산(峨山)이라는 그의 아호는 자신의 출생지 옛 지명에서 따온 것이다. 1930년 통천 송전소학교를 '
 '졸업하였고 그와 함께한 동창생은 27명이며 정주영의 최종 학력은 소학교(초등학교) 졸업 이 유일하다. \n'
 '상황이 이렇다 보니 프로축구 현대 호랑이는  홈 앤 어웨이 제도로 바뀐 한편 주말 2연전이 새롭게 도입된  동시에 프로야구의 선례를 본따 '
 '홈팀이 흰색,원정팀이 유색 유니폼을 입은   1987년  본인(정주영)의 고향인 강원도를 연고지로 설정했다가  1990년 울산으로 '
 '연고지를 옮겼으며  주말 2연전 제도는 선수 부상 등의  문제점이 생기자 1년 만에 폐지됐고  다음 해부터 홈팀은 유색, 원정팀은 흰색 '
 '유니폼을 입는 것으로 변경됐다.')


# train 데이터셋의 context 데이터 탐색
- `train` 데이터셋의 context 대한 탐색 내용입니다
- 살펴본 내용은 다음과 같습니다.

1. 데이터 기본 정보 확인
    - 확인된 전처리 필요한 문자
2. 데이터 중복 확인 및 데이터 탐색
    - document_id 기준으로 중복 확인
    - title 기준으로 중복 확인
3. language 종류 확인
4. 데이터 전처리 필요 부분 확인 (문법, 어휘, 특수 문자 등)
5. question과 answer에도 다른 언어가 존재할까?
6. 년도, 해 로 끝나는 질문에 대한 답변

In [ ]:
# 데이터셋 부르기 위한 datasets 라이브러리 다운로드
#pip install datasets

In [2]:
# 탐색에 필요한 라이브러리 import
from datasets import load_from_disk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from collections import Counter
import re

In [3]:
# pandas datasframe으로 변환 및 필요한 컬럼만 추출하는 함수
def dataset_to_df(dataset):
  data_list = []

  for data in dataset:
    data_list.append({
        'title' : data["title"],
        "context" : data["context"],
        "question" : data["question"],
        "answer_start" : data["answers"]["answer_start"][0],
        "answer_text" : data["answers"]["text"][0],
        "document_id" : data["document_id"]
    })
  df = pd.DataFrame(data_list)
  return df

In [4]:
# 데이터셋 불러오기
path = "your path"
dataset = load_from_disk(path)
print(dataset)
print(dataset['train'])

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})
Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 3952
})


In [5]:
train_df = dataset_to_df(dataset["train"])
train_df.head()

,title,context,question,answer_start,answer_text,document_id
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,235,하원,18293
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,212,《경영의 실제》,51638
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,510,백성,5028
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,625,중국,34146
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,30,4개,47334


### 1.데이터 기본 정보 확인

In [6]:
train_df.head()

,title,context,question,answer_start,answer_text,document_id
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,235,하원,18293
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,212,《경영의 실제》,51638
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,510,백성,5028
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,625,중국,34146
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,30,4개,47334


In [7]:
# 데이터의 개수
print(f"데이터 개수 : {len(train_df)}")

데이터 개수 : 3952


In [ ]:
# 랜덤 샘플링한 데이터
# 전체 데이터 중 200개를 랜덤으로 뽑기
sample_context = random.sample(list(train_df["context"]), 200)

print(sample_context)

In [ ]:
# 랜덤 샘플된 데이터 직접 확인하기
for context in sample_context:
  print(context)
  print("=" * 50)

- 확인된 전처리 필요한 문자 <br>
  - `\\n`, `\n`, `\n*`, `\\n\\n`, `\n\n`

- 확인된 언어의 종류<br>
  1. 한국어
  2. 영어
  3. 일본어
  4. 프랑스어
  5. 한자
  6. 아랍어

- 제거가 필요한지 애매한 문자<br>
  - `가우스 그룹#s-1.3`
  - 자세한 내용은 [confluence](https://) 참고

- 특이한 특수 문자
  - `△사용자`

### 2.데이터 중복 확인 및 데이터 탐색


In [10]:
# 특정 칼럼을 기준으로 중복된 값을 Counter 함수로 찾은 뒤 리턴하는 함수
def count_duplication(column):
  count = dict(Counter(train_df[column]))
  count = sorted(count.items(),
                 reverse=True,
                 key=lambda item:item[1])

  return count

In [11]:
def print_duplication(column):
  total = len(train_df)
  unique_column = len(train_df[column].unique())

  print(f"전체 데이터 개수 : {total}")
  print(f"unique한 {column} 개수 : {unique_column}")

  if total != unique_column:
    print(f"중복된 값이 있습니다. 중복된 값 개수 : {total - unique_column}")
  else:
    print("중복된 값이 없습니다.")

#### document_id 기준으로 중복 확인


In [12]:
# document_id 중복 여부
print_duplication("document_id")

전체 데이터 개수 : 3952
unique한 document_id 개수 : 3340
중복된 값이 있습니다. 중복된 값 개수 : 612


In [ ]:
document_count = count_duplication("document_id")
print(document_count)

In [14]:
train_df[train_df["document_id"] == document_count[0][0]]

,title,context,question,answer_start,answer_text,document_id
201,윤치호,"그러나 한편으로는 미국 사회의 인종차별주의적, 흑인을 차별하는 태도를 목격하면서 백...",평상시 기독교선교사들로부터 차별적 대우를 받은 윤치호가 생각한 미국의 중대한 속성은?,645,인종주의,5284
1426,윤치호,"그러나 한편으로는 미국 사회의 인종차별주의적, 흑인을 차별하는 태도를 목격하면서 백...",윤치호가 미국의 특징이 인종주의라고 생각하게 만든 인물은?,570,기독교선교사들,5284
1622,윤치호,"그러나 한편으로는 미국 사회의 인종차별주의적, 흑인을 차별하는 태도를 목격하면서 백...",조선에서도 윤치호를 유색 인종이라는 이유로 차별한 사람은?,254,미국인 선교사,5284
2199,윤치호,"그러나 한편으로는 미국 사회의 인종차별주의적, 흑인을 차별하는 태도를 목격하면서 백...",윤치호는 누구의 설교를 듣고 충격을 받았나?,824,남부 출신 남감리교회 목사들,5284


In [15]:
train_df[train_df["document_id"] == document_count[3][0]]

,title,context,question,answer_start,answer_text,document_id
110,빌 클린턴,클린턴의 첫 기간의 말기가 접근하면서 새로운 스캔들이 일어났다. 스캔들은 클린턴과 ...,클린턴의 두 번째 임기는 어떤 스캔들로 인해 내리막을 걷게 되었나?,257,화이트워터 사건,4633
3034,빌 클린턴,클린턴의 첫 기간의 말기가 접근하면서 새로운 스캔들이 일어났다. 스캔들은 클린턴과 ...,클린턴은 1996년 재선에서 누구를 꺾고 선거를 이겼나요?,188,자신의 공화당 상대 후보 밥 돌,4633
3818,빌 클린턴,클린턴의 첫 기간의 말기가 접근하면서 새로운 스캔들이 일어났다. 스캔들은 클린턴과 ...,두 번째 선거에서 빌 클린턴은 누구를 상대로 승리하였나?,202,밥 돌,4633


#### title 기준으로 중복 확인

In [16]:
# title 중복 여부 확인
print_duplication("title")

전체 데이터 개수 : 3952
unique한 title 개수 : 2716
중복된 값이 있습니다. 중복된 값 개수 : 1236


In [ ]:
# title 중복 개수 확인
title_count = count_duplication("title")
print(title_count)

In [ ]:
train_df[train_df["title"] == title_count[0][0]]

In [19]:
train_df[(train_df["title"] == title_count[0][0]) & (train_df["answer_text"] == "다산 정약용")]

,title,context,question,answer_start,answer_text,document_id
19,윤치호,"혹자는 윤치호의 직계선조인 윤근수, 윤두수로 그의 문중은 당색으로는 서인 소론 계열...",윤치호가 극찬했던 조선시대의 인물은 누구인가요?,585,다산 정약용,5322
1212,윤치호,"혹자는 윤치호의 직계선조인 윤근수, 윤두수로 그의 문중은 당색으로는 서인 소론 계열...",천주교로 종교를 바꿨다는 의심을 받았던 학자는?,300,다산 정약용,5322
2442,윤치호,"혹자는 윤치호의 직계선조인 윤근수, 윤두수로 그의 문중은 당색으로는 서인 소론 계열...",윤치호가 조선에서 탄생한 위대한 인물이라 칭하며 극찬을 보낸 인물은?,585,다산 정약용,5322


#### document_id가 같으면서 answer_start가 같은지 확인

In [ ]:
document_count = count_duplication("document_id")
print(document_count)

In [21]:
# document_id가 같으면서 answer_start가 같은 데이터 추출, document_id 값 추출
document_list = []
for count in document_count:
  if count[1] > 1:
    document_count_df = train_df[(train_df["document_id"] == count[0])]
    same_answer_start_df = document_count_df[document_count_df.duplicated('answer_start', keep=False)]
    if len(same_answer_start_df) > 0:
      document_list.append(count[0])

print(document_list)

[5322, 8428, 7623, 6653, 8190, 6716, 7139, 7833, 25068, 49393, 15663, 33391, 14917, 53065, 5031, 57931, 36795, 5282, 35777, 6711, 51137, 6907, 33784, 47393, 47169, 6329, 32475, 6541, 20027, 10686, 6773]


In [22]:
document_count_df = train_df[(train_df["document_id"] == 5322)]
same_answer_start_df = document_count_df[document_count_df.duplicated('answer_start', keep=False)]
same_answer_start_df

,title,context,question,answer_start,answer_text,document_id
19,윤치호,"혹자는 윤치호의 직계선조인 윤근수, 윤두수로 그의 문중은 당색으로는 서인 소론 계열...",윤치호가 극찬했던 조선시대의 인물은 누구인가요?,585,다산 정약용,5322
2442,윤치호,"혹자는 윤치호의 직계선조인 윤근수, 윤두수로 그의 문중은 당색으로는 서인 소론 계열...",윤치호가 조선에서 탄생한 위대한 인물이라 칭하며 극찬을 보낸 인물은?,585,다산 정약용,5322


In [23]:
document_answer_df = pd.DataFrame()
for document in document_list:
  document_count_df = train_df[(train_df["document_id"] == document)]
  same_answer_start_df = document_count_df[document_count_df.duplicated('answer_start', keep=False)]
  document_answer_df = pd.concat([document_answer_df, same_answer_start_df], ignore_index=True)

document_answer_df.head(10)

,title,context,question,answer_start,answer_text,document_id
0,윤치호,"혹자는 윤치호의 직계선조인 윤근수, 윤두수로 그의 문중은 당색으로는 서인 소론 계열...",윤치호가 극찬했던 조선시대의 인물은 누구인가요?,585,다산 정약용,5322
1,윤치호,"혹자는 윤치호의 직계선조인 윤근수, 윤두수로 그의 문중은 당색으로는 서인 소론 계열...",윤치호가 조선에서 탄생한 위대한 인물이라 칭하며 극찬을 보낸 인물은?,585,다산 정약용,5322
2,선언형 프로그래밍,"선언형 프로그래밍은 두 가지 뜻으로 통용되고 있다.\n\n한 정의에 따르면, 프로그...",선언형 프로그램과 대비되는 프로그램은?,328,명령형 프로그램,8428
3,선언형 프로그래밍,"선언형 프로그래밍은 두 가지 뜻으로 통용되고 있다.\n\n한 정의에 따르면, 프로그...",선언형 프로그램의 알고리즘에 대한 정의와 반대인 개념은?,328,명령형 프로그램,8428
4,꿀풀과,"꿀풀과(--科, 라미아케아이)는 꿀풀목의 과이다. 꿀풀목 가운데 가장 큰 과이며 전...",꿀풀목 가운데 가장 큰 과와 생김새가 비슷한 과는?,562,현삼과,7623
5,꿀풀과,"꿀풀과(--科, 라미아케아이)는 꿀풀목의 과이다. 꿀풀목 가운데 가장 큰 과이며 전...",꿀풀과와 유사하게 생긴 식물은?,562,현삼과,7623
6,꿀풀과,"꿀풀과(--科, 라미아케아이)는 꿀풀목의 과이다. 꿀풀목 가운데 가장 큰 과이며 전...",분과에 한 개의 씨가 들어있지 않은 풀의 종류는?,562,현삼,7623
7,모아이,"모아이(Moai, Moái, 모이아스)는 칠레 이스터 섬에 있는 사람 얼굴 모양의 ...",모아이는 무엇으로 이루어져있나?,445,화산암,6653
8,모아이,"모아이(Moai, Moái, 모이아스)는 칠레 이스터 섬에 있는 사람 얼굴 모양의 ...",모아이를 이루는 돌의 종류는 무엇인가?,445,화산암,6653
9,베트남의 역사,베트남 전쟁\n\n 베트남 전쟁은 1964년 통킹만 사건을 구실로 미국이 북 베트남...,제2차 인도차이나 전쟁이 끝났던 해는?,131,1975년,8190


### 3.language 종류 확인

In [24]:
language_train_df = train_df
language_train_df.head()

,title,context,question,answer_start,answer_text,document_id
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,235,하원,18293
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,212,《경영의 실제》,51638
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,510,백성,5028
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,625,중국,34146
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,30,4개,47334


In [25]:
# 언어 분류 함수
def classify_language(text):
    has_english = False
    has_korean = False
    has_japanese = False
    has_chinese = False
    has_arabic = False
    has_other = False

    for char in text:
        # 영어 (A-Z, a-z)
        if 'A' <= char <= 'Z' or 'a' <= char <= 'z':
            has_english = True
        # 한글 (유니코드 범위: U+AC00 ~ U+D7A3)
        elif '가' <= char <= '힣':
            has_korean = True
        # 일본어 히라가나 (U+3040 ~ U+309F) 및 가타카나 (U+30A0 ~ U+30FF)
        elif '\u3040' <= char <= '\u309F' or '\u30A0' <= char <= '\u30FF':
            has_japanese = True
        # 한자 (유니코드 범위: U+4E00 ~ U+9FFF)
        elif '\u4E00' <= char <= '\u9FFF':
            has_chinese = True
        # 아랍어 (유니코드 범위: U+0600 ~ U+06FF)
        elif '\u0600' <= char <= '\u06FF':
            has_arabic = True
        # 기타 문자
        else:
            has_other = True

    # 결과 반환
    languages = []
    if has_korean:
        languages.append("Korean")
    if has_english:
        languages.append("English")
    if has_japanese:
        languages.append("Japanese")
    if has_chinese:
        languages.append("Chinese")
    if has_arabic:
        languages.append("Arabic")
    if has_other and not languages:
        languages.append("Other")

    return ", ".join(languages) if languages else "Other"

# 'context' 열에서 언어 분석 후 새로운 열 추가
language_train_df['languages'] = language_train_df['context'].apply(classify_language)

# 결과 출력
print(language_train_df[['context', 'languages']])

                                                context  \
0     미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...   
1     '근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...   
2     강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...   
3     불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...   
4     동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...   
...                                                 ...   
3947  이오의 산\n이오의 산 목록\n 이오에는 100~150개의 산이 있다. 이들 산의 ...   
3948  애니의 고군분투 뉴욕 입성기!!\n\n인류학자가 꿈인 21살 소녀 '애니(스칼렛 요...   
3949  1842년에 작곡가이자 지휘자인 오토 니콜라이가 빈 궁정 오페라극장 소속 관현악단을...   
3950  원어는 고대 그리스어까지 거슬러 올라간다. 영어 문헌에 이 말이 나타나기 시작한 것...   
3951  2008년 2월 28일 실시된 2008년 함부르크 주의회 선거에서 기민련은 과반수 ...   

                     languages  
0              Korean, English  
1              Korean, English  
2     Korean, English, Chinese  
3     Korean, English, Chinese  
4     Korean, English, Chinese  
...                        ...  
3947  Korean, English, Chinese  
3948           Korean, Engl

In [26]:
language_train_df.head(10)

,title,context,question,answer_start,answer_text,document_id,languages
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,235,하원,18293,"Korean, English"
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,212,《경영의 실제》,51638,"Korean, English"
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,510,백성,5028,"Korean, English, Chinese"
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,625,중국,34146,"Korean, English, Chinese"
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,30,4개,47334,"Korean, English, Chinese"
5,용아병,델포이의 신탁에 따라 암소를 따라간 카드모스는 테베 땅에 이르렀다. 카드모스는 암소...,카드모스의 부하들이 간 곳에는 무엇이 있었는가?,91,드래곤,37482,"Korean, English"
6,호반,"제27회와 제73회에 출현한다. 관우는 자신을 저지하는 공수, 맹탄, 한복, 변희를...",관우를 불태워 죽이려한 사람 누구인가?,68,형양태수 왕식,24395,"Korean, English, Chinese"
7,제50노섬브리안 보병사단,이집트로 이어지는 해안 도로는 1개의 사단만이 방어할 수 있었으며 엘 아뎀 요새와 ...,참호 속에 무기와 장비를 버리고 도주한 집단은?,583,이탈리아군,43399,"Korean, English"
8,오싱,하지만 슬픈 일들이 그녀의 행복을 질시하고 있었다.\n평생 고생만 한 어머니를 백혈...,제2차 세계 대전에 참전하여 사망한 자식은?,195,큰아들 유,10426,"Korean, English, Chinese"
9,모후산,모후산(母后山)대한민국 전라남도 화순군과 순천시의 경계를 이루는 높이 943.7m의...,고려 공민왕이 처가 식구들과 아내와 함께 피신처로 삼은 마을은?,861,왕대마을,15766,"Korean, English, Chinese"


In [27]:
# context 데이터에 사용된 언어 종류 확인
unique_languages = set()

for language in list(language_train_df["languages"].unique()):
    languages = language.split(', ')
    unique_languages.update(languages)

result = list(unique_languages)

print(result)

['Korean', 'Arabic', 'English', 'Chinese', 'Japanese']


In [ ]:
contains_df = language_train_df[language_train_df["context"].str.contains("폴 르 노르망")]
print(contains_df.to_string())

### 4.데이터 전처리 필요 부분 확인 (문법, 어휘, 특수 문자 등)


In [29]:
# 개행 문자 카운트 함수 (by 정휘님)
def get_newline(data_samples):
    all_lines = []
    for i,data in enumerate(data_samples):
        new_line_count = data.count("\\n")
        all_lines.append([i, new_line_count])
    return all_lines

In [30]:
newline_counts = np.array(get_newline(train_df["context"]))
print(np.sum(newline_counts[:,1],axis=0))

22808


In [31]:
# 특수 문자를 찾는 함수 정의
def find_special_characters(text, pattern):
    # 정규 표현식을 사용하여 특수 문자 찾기
    return re.findall(pattern, text)

# context에서 모든 특수 문자를 찾고 중복 제거
all_special_chars = set()
#pattern = r'[^a-zA-Z0-9가-힣\s]' # 알파벳, 숫자, 한글, 공백 제외
#pattern = r'[^a-zA-Z0-9가-힣\s\u4e00-\u9fff]' # 알파벳, 숫자, 한글, 한자 공백 제외
pattern = r'[^a-zA-Z0-9가-힣\s\u4e00-\u9FFF\u3040-\u309F\u30A0-\u30FF\u0600-\u06FF]' # 알파벳, 숫자, 한글, 한자, 일본어, 아랍어 공백 제외
for context in train_df['context']:
    special_chars = find_special_characters(context, pattern)
    all_special_chars.update(special_chars)

# 결과 출력
print(f"찾은 특수 문자 종류: {all_special_chars}")
print(f"특수 문자의 개수: {len(all_special_chars)}")

찾은 특수 문자 종류: {'া', 'υ', 'о', 'θ', '°', 'Н', '③', '＞', 'Þ', 'µ', 'η', 'н', 'ழ', 'е', 'ü', 'क', '“', '\u200e', 'ㆍ', '+', '『', '⑦', 'р', 'ό', 'ɔ', 'ׇ', 'ா', '}', 'ㅋ', '→', 'ˈ', 'І', '△', 'ர', 'ㅁ', '±', 'Α', 'ნ', 'ː', '㎞', 'ㄷ', 'ᛁ', 'μ', ',', '←', 'ე', '３', 'ă', '〕', 'α', 'ი', '】', '。', 'ŭ', '㎜', '＼', '"', '·', '１', 'в', 'ν', ':', 'ि', 'י', 'ὔ', '…', 'Ｖ', 'π', 'ə', '―', '’', '(', '☆', 'ờ', '（', 'é', '▲', 'Κ', '&', 'з', ']', '□', "'", 'ᛚ', 'с', 'Φ', 'ा', 'ú', 'ய', '\xad', '⑪', '\uf000', 'გ', 'त', 'ㅅ', 'დ', 'ს', 'м', '○', '*', '#', 'Ｍ', 'ㄹ', 'к', '８', 'á', '５', 'æ', 'ê', 'ㅕ', '/', 'ㅏ', 'ć', '≫', 'ί', 'ǫ', 'ξ', 'ვ', 'ý', 'ɛ', 'γ', 'ð', ')', 'Р', '.', '㎠', 'ε', 'ㅚ', '～', '↓', '♣', 'ო', 'Δ', 'ώ', 'í', 'É', '？', 'ㄱ', 'ㅎ', 'ხ', 'ஜ', 'ც', '∧', 'ó', '_', '⑤', 'й', 'Ε', 'ơ', '⑥', '²', 'х', '—', '－', 'ω', 'ë', 'В', '%', '$', '்', '`', 'ē', '́', 'ल', '㎝', 'ַ', '４', 'έ', '」', 'ч', 'і', 'ì', 'à', 'β', 'Ｘ', 'Ἰ', 'л', 'Λ', 'ש', 'χ', 'Π', 'ோ', '×', 'κ', '?', 'ㄴ', '℃', '|', 'ğ', 'œ', '【', '〔', 'ά', '¾', 'স'

In [32]:
pd.set_option("display.max_colwidth", None)

In [33]:
train_df[train_df["context"].str.contains("\uf000")]["context"]

,context
961,"구포(龜浦)라는 지명은 거북龜자로 시작되므로 거북이에 대한 이야기가 전해 내려오고 있다. 그 중에서 《양산군지(梁山郡誌)》에 나오는 구포에 관한 기록은 “창(倉)“을 설치하였으므로 남창(南倉)이라고 하며 범방산(泛舫山) 한 줄기가 낙동강을 향하여 머리에 돌을 이고 있는 모습이 거북이와 같다.는 연유에서 구포 지명의 유래를 밝히고 있다.\n\n여기에 나오는 범방산은 백양산(운수산) 줄기로서 강변쪽으로 뻗어내린 구포의 남쪽 구남마을 뒷산을 말한다. 현재 모라동 쪽에서 구포도서관, 백양고, 구포중, 모라중이 들어 서 있는 곳으로 뻗어내려 구포초등학교까지 이어지는 산이다.\n\n이 산의 형상을 강 건너쪽 멀리서 바라보면 거북이의 머리와 몸체를 그대로 빼어 놓은 것처럼 닮아 보인다. 범방산 능선에는 온갖 바위들이 소나무와 얽혀 군집해 있는데 《양산군지》의 기록처럼 머리에 돌을 이고 있는 듯한 모습을 보이고 있는 것이다. 그리고 범방산을 구포 본 동네에서 쳐다보면 산등성이에 거북이 형상을 한 큰 바위가 산으로 오르는 모습을 하고 있다.\n\n이처럼 거북이 형상을 한 산줄기에 거북바위까지 있어 구포사람들은 이 산을 거북산이라고 부르는데 구포의 지명이 여기에서 왔다는 것이다. 옛날 바다의 거북이가 겨울에도 따뜻한 기운이 있어 이를 보고 사람들이 바다의 거북이가 물가 모래밭에 구멍을 파고 겨울잠을 자면서 쉬어가는 곳이다 하여 구포의 지명에 유래를 들고 있다. 그래서 거북이가 쉬어가는 갯가로서 ""거부개""즉 ""龜浦""라는 것이다.\n\n조선시대 구포를 감동진, 감동나루로 불러왔는데 이 감동진나루는 낙동강 3대 나루터의 한 곳으로 수로교통과 교역의 시발지로서 강변 언덕위에는 정부의 세곡(稅穀)을 보관하는 남창(南倉)이 설치되어 있었다. 그래서 구포 감동나루가 있는 이곳의 행정지명이 양산군 좌이면\n남창리로 나와 있다.\n\n남창을 배경으로 강변에서부터 안쪽 넓은 공터에서 구포장(場)이 섰는데 장타령에 나올만큼 큰 장터였고 곡물선과 상선, 어선들이 드나들면서 크게 번창했는데 근세에 이르기까지 낙동강 수로교통의 시발지로서 구포는 널리 알려져 왔다."


In [34]:
special_char = "⑥"
train_df[train_df["context"].str.contains(special_char)]["context"]

,context
308,"구매품의 수량 결정은 연속생산을 위한 상비품인가, 주문생산을 위한 주문품인가에 따라 다르다. 즉 주문품의 경우 요구량을 기준으로 하지만, 상비품의 경우는 주문량에 의한 단가나 단위당 구매비용이 저렴하고, 저장비용을 늘여서 자금고정을 크게 하는 것을 고려, 구매 및 저장에 따르는 원가를 우선 싸게 계획해야 한다. 구매시기는 수량에 따라 결정된다.\n구매수량 및 시기결정에 있어 고려해야 할 요소는 다음과 같다. ① 총재고량 및 유효재고량, ② 사용량은 계속사용·임시사용·계절적 사용을 고려한다. ③ 발주에서 납품까지에 소요되는 기간, ④ 납기지연에 대비하는 예비량, ⑤ 수요변동에 대비하는 긴급수량, ⑥ 자재보유기간, ⑦ 대량매입으로 유리하게 되는 양, ⑧ 투입자금과 그 효율, ⑨ 기타 자재획득에 소요되는 비용, 예를 들면 운임·수수료·보험료 등이다.\n〔그림 2〕는 구매수량 및 시기와 재고량·사용량 등의 상호관계를 나타내는데, 이 그림에 의해 최대·최소 재고량, 평균사용량 및 조달기간이 주어지면 적정주문량·발주시기가 결정된다.\n또한 구매에 있어 단위당 '주문비용'과 '저장비용'이 같아지기 때문에 다음과 같은 관계가 성립되며, 이로써 경제적 주문량()이 구해진다.\n：단위당 주문비용 ：단위당 저장비용\n：경제적 주문량 ：1단위당 원가\n：재고비용(이자·보험료 등의 율)\n：연 구매량(또는 사용량)"
1018,"일반 상대성 이론\n\n일반 상대성 이론은 1915년에 아인슈타인이 발표하였다. (프로이센 과학 아카데미에서 1915년 11월 25일에 제출) 일반 상대성이론은 특수 상대성이론의 확장판이라 하면 이해하기 쉬울 것이다. 이 이론은 뉴턴의 고전 물리학에 결정타를 날림으로서 새로운 물리학적 이론의 길을 열었다는 점에서 의의가 있다고 할 수 있다.\n\n일반상대성이론에 대해 알아보기 전에 먼저 이를 전개하기 위해 필요한 한 가지 가정을 보도록 하자.\n\n등가원리\n-가속 좌표계에서 지구로 인해 생기는 중력과 중력가속도g와 같은 크기의 가속도 a로 중력의 작용방향과 반대로 운동하는 것으로 인한 관성력은 구분할 수 없다.\n\n즉, 관성질량과 중력질량이 같은 측정값을 지닌다는 것이다.\n\n1. 시공간\n일반 상대성 이론에서 아인슈타인은 '시공간은 4차원, 즉 시간과 공간이 결합된 형태의 연속체'임을 규명했다. 뉴턴이 시공간이 시간과 공간으로 분리되어 서로 영향을 주고받지 않는 별개의 공간인 점과, 공간위의 한 점에 위치한 물체에 어떠한 영향도 받지 않을 것이라고 생각한 반면에 아인슈타인의 사고실험에서 아인슈타인은 시공간이 서로 상호작용함과, 공간위에 위치한 어떠한 질량을 가진 물체에 대하여 공간이 휘어짐을 제시하였다.\n\n2. 중력\n아인슈타인은 뉴턴의 중력을 가속운동계에 적용시켰다. 그는 중력가속도g의 크기만큼 가속하는 가속계 내의 물체와, 중력을 받고있는 물체는 서로 구분이 불가능하다는 이론을 내세움으로서 가속계를 관성계로 해석가능함을 보였다.\n\n3. 중력으로 인한 시간팽창\n위에서 가속계 역시 관성계로 인식할 수 있다고 하였다. 따라서 가속계의 물체는 관성계에서 물체의 이동으로 판단할 수 있고, 특수 상대성이론에 따라 시간지연이 일어남을 알 수 있다. 이로서 중력을 받는 물체는 그 물체에 흐르는 시간이 느려지게 된다.\n\n4. 공간 왜곡으로 일어나는 현상\n4-1. 빛의 휘어짐 : 빛의 움직임을 가속운동계에 적용시켜보면, 그 빛이 휜다는 것을 알 수 있을 것이다. 따라서 중력으로 인하여 빛이 휜다는 것을 알 수 있다\n\n4-2. 중력렌즈효과 : 이는 일식이 일어날 때 관측할 수 있는 현상인데, 바로 태양의 뒤에 위치하여 가려져 있던 천체가 태양의 중력에 의한 영향으로 그 빛이 휘어져 우리눈에 들어오는 것이다. 흔히 '아인슈타인의 십자가'라는 것이 이로 인한 현상이다.\n\n4-3. 블랙홀 : 질량이 매우 큰 천체는 공간을 심하게 왜곡하여 빛마저 삼켜버릴 수 있다.\n\n5. 일반상대성이론의 증거\n① 에딩턴의 태양의 일식 관측 : 지구에서 별을 관측할 때, 별과 지구사이에 태양이 있을 때와 없을 때의 별의 관측위치를 비교하여 빛이 휘는 것을 관찰\n\n② 수성의 세차운동 : 뉴턴의 이론에 따라 계산하면 100년동안 574´´만큼 이동해야 하지만, 실제 세차운동 관측결과 43´´정도 오차가 났다. 하지만 일반상대성 이론에 시공간의 곡률을 고려한 결과 이 차이를 정확히 설명\n\n③ 중력렌즈 : 중력이 렌즈처럼 빛을 휘게 하는 현상이다. 대표적으로 퀘이사가 있다.(퀘이사는 은하의 중력 때문에 지구에서 4개의 빛나는 쌍둥이별로 오인했었다.)\n\n④ 중력파 : 천체의 중력붕괴나 초신성폭발 같은 우주현상으로 발생하여 시공간이 일그러짐이 광속으로 파도처럼 전달되는 것을 말한다.(이전까지 발견되지 않고있다가 최근 블랙홀에서 발견되었다.)\n\n⑤ GPS : 인공위성이 움직이기 때문에 시간차이가 생기는데 속력과 중력의 작용을 고려하여 보정해줘야 한다.\n\n⑥ 블랙홀 : 질량이 매우 큰 천체는 공간을 휘게하여 천체를 지나는 빛마저 흡수한다.(아인슈타인이 2차원의 평면에 시간의 곡률을 표기한 그림에서 보면 블랙홀은 질량이 극도로 커 평면 자체가 엄청나게 움푹 들어가있다. 이때 블랙홀의 부분중 넘게되면 빛조차 절대 빠져나올수 없는 선이있는데 이곳을 사건의 지평선이라고 하며 이 안쪽부분은 어떠한것도 빠져나오지 못하므로 검게보인다.)"
3805,"일반 상대성 이론\n\n일반 상대성 이론은 1915년에 아인슈타인이 발표하였다. (프로이센 과학 아카데미에서 1915년 11월 25일에 제출) 일반 상대성이론은 특수 상대성이론의 확장판이라 하면 이해하기 쉬울 것이다. 이 이론은 뉴턴의 고전 물리학에 결정타를 날림으로서 새로운 물리학적 이론의 길을 열었다는 점에서 의의가 있다고 할 수 있다.\n\n일반상대성이론에 대해 알아보기 전에 먼저 이를 전개하기 위해 필요한 한 가지 가정을 보도록 하자.\n\n등가원리\n-가속 좌표계에서 지구로 인해 생기는 중력과 중력가속도g와 같은 크기의 가속도 a로 중력의 작용방향과 반대로 운동하는 것으로 인한 관성력은 구분할 수 없다.\n\n즉, 관성질량과 중력질량이 같은 측정값을 지닌다는 것이다.\n\n1. 시공간\n일반 상대성 이론에서 아인슈타인은 '시공간은 4차원, 즉 시간과 공간이 결합된 형태의 연속체'임을 규명했다. 뉴턴이 시공간이 시간과 공간으로 분리되어 서로 영향을 주고받지 않는 별개의 공간인 점과, 공간위의 한 점에 위치한 물체에 어떠한 영향도 받지 않을 것이라고 생각한 반면에 아인슈타인의 사고실험에서 아인슈타인은 시공간이 서로 상호작용함과, 공간위에 위치한 어떠한 질량을 가진 물체에 대하여 공간이 휘어짐을 제시하였다.\n\n2. 중력\n아인슈타인은 뉴턴의 중력을 가속운동계에 적용시켰다. 그는 중력가속도g의 크기만큼 가속하는 가속계 내의 물체와, 중력을 받고있는 물체는 서로 구분이 불가능하다는 이론을 내세움으로서 가속계를 관성계로 해석가능함을 보였다.\n\n3. 중력으로 인한 시간팽창\n위에서 가속계 역시 관성계로 인식할 수 있다고 하였다. 따라서 가속계의 물체는 관성계에서 물체의 이동으로 판단할 수 있고, 특수 상대성이론에 따라 시간지연이 일어남을 알 수 있다. 이로서 중력을 받는 물체는 그 물체에 흐르는 시간이 느려지게 된다.\n\n4. 공간 왜곡으로 일어나는 현상\n4-1. 빛의 휘어짐 : 빛의 움직임을 가속운동계에 적용시켜보면, 그 빛이 휜다는 것을 알 수 있을 것이다. 따라서 중력으로 인하여 빛이 휜다는 것을 알 수 있다\n\n4-2. 중력렌즈효과 : 이는 일식이 일어날 때 관측할 수 있는 현상인데, 바로 태양의 뒤에 위치하여 가려져 있던 천체가 태양의 중력에 의한 영향으로 그 빛이 휘어져 우리눈에 들어오는 것이다. 흔히 '아인슈타인의 십자가'라는 것이 이로 인한 현상이다.\n\n4-3. 블랙홀 : 질량이 매우 큰 천체는 공간을 심하게 왜곡하여 빛마저 삼켜버릴 수 있다.\n\n5. 일반상대성이론의 증거\n① 에딩턴의 태양의 일식 관측 : 지구에서 별을 관측할 때, 별과 지구사이에 태양이 있을 때와 없을 때의 별의 관측위치를 비교하여 빛이 휘는 것을 관찰\n\n② 수성의

In [35]:
special_char = "ä"
train_df[train_df["context"].str.contains(special_char)]["context"]

,context
1144,"아른험에 도착하자 사단의 재편성이 시작되었다. 잔존한 기갑 차량 대부분이 독일 본토의 정비공장으로 운송되기 위해 기차에 선적되었다. 9월 17일 일요일, 연합군의 마켓가든 작전이 개시되었고 영국 제1 공수 사단이 아른험 서쪽 오스테르베이크(Oosterbeek)에 강하했다. 위험을 감지한 제2 SS 기갑 군단장 비트리히 SS대장은 호엔슈타우펜과 프룬츠베르크에 임전태세를 취하라는 명령을 하달했다. 사단의 기갑 차량들이 기차에서 하역되었고, 정비 부대는 기갑 차량들의 궤도간격을 재조정하기 위해 미친듯이 작업했다. 사단 기갑 부대 중 오직 정찰 부대인 제9 SS 기갑수색 대대만이 구륜 및 반궤도 차량을 정수에 가깝게 보유하고 있었기에 즉시 투입이 가능했다.\n\n비트리히 SS대장은 호엔슈타우펜에 아른험을 점거하고 중요 지물인 아른험 교량을 확보하라는 명령을 하달했다. 하르처 SS중령은 사단을 아른험에 투입했고, 그들이 로텐 토이펠(Roten Teufel = 붉은 악마)이라 부른 영군 강하병들의 강력한 저항에 부딪혔다. 파울 그레브너(Paul Gräbner) SS대위가 지휘하는 기갑수색 대대가 네이메헌(Nijmegen) 부근으로 정찰을 목적으로 투입되었다. 그레브너 SS대위는 노르망디에서의 활약으로 당일 기사십자장을 수여받았다.\n\n기갑수색 대대가 아른험 남쪽에서 정찰을 하고 있을 무렵, 존 프로스트(John Frost) 중령의 영국 제1 공수 여단 2대대가 아른험으로 진입했고 다리 북쪽 끝에 방어 진지를 구축했다. 9월 18일 아침, 그레브너는 남쪽에서 정찰 임무를 마치고 복귀해 아른험 다리를 확보하라는 명령을 하달받았다.\n\n그레브너의 무리한 계획은 수수께끼로 남아있지만, 확실한 건 그레브너가 다리를 탈환하거나 혹은 영국군의 방어선을 돌파해 아른험 수비에 투입된 사단 본대를 지원하려고 했다는 것이다. 어쨌든지 간에 그레브너 SS대위의 공격 시도는 처참하게 끝이 났다. 태세를 갖춘 영군 강하병들은 선두 차량 4량을 무사히 보낸 후 대전차 화기인 피아트 대전차 로켓, 화염방사기 그리고 소화기들로 공격을 개시했다. 2시간의 전투로 호엔슈타우펜 기갑수색 대대는 실질적으로 괴멸되었으며, 그로 인해 그레브너 SS대위를 포함해 약 70명이 전사했고 차량 22량이 파괴되었다. 머나먼 다리에 이 장면이 묘사되어 있다.\n\n8일에 걸쳐 벌어진 전투에 사단은 하루도 빠짐없이 참가했으며, 프로스트 중령의 대대와 아른험 서쪽에서 전투를 치렀다. 또한 오스테르베이크 부근에서 포위된 제1 공수 사단 본대를 지속적으로 압박했다. 아른험 전투는 호엔슈타우펜의 대승이었다. 여타 독일군 부대들의 지원을 바탕으로 피폐해진 사단이 정예 영국 공수 부대를 괴멸시킨 것이다. 격렬한 전투에도 불구하고 호엔슈타우펜 및 프룬츠베르크 장병들은 포로가 된 강하병들을 정중하게 대해주었으며, 비트리히 SS대장은 소련군조차도 능가하는 붉은 악마의 끈기와 전투력에 주목했다.\n\n마켓가든 작전의 패인이 ""하필이면 아른험에 주둔한"" 제9 SS기갑 사단 및 제10 SS기갑 사단 때문이라는 것이 기존 학설이었고, 영화 머나먼 다리 역시 코넬리우스 라이언의 동명 저서도 이 학설에 기초한 것이었다. 그러나 90년대 이후 새롭게 제기된 학설은 당시 제9 SS기갑 사단 및 제10 SS기갑 사단의 전력은 이름만 기갑사단일 뿐 전차는 거의 보유하지 않았으며, 앞서 언급된 그레브너 SS대위의 기갑수색 대대가 가용한 기갑 전력의 거의 전부였다는 사실이 밝혀졌다. 아울러 이 무렵 두 사단의 병력은 합쳐도 1개 사단에 미치지 못하는 수준이었다는 것이다. 아울러 마켓가든 작전의 실패 책임을 이 두 사단으로 돌려서 영국군이 저지른 실책과 제1 공수 사단의 사실상 전멸에 대한 책임을 모면하려는 것이라고 주장하기도 한다."
1536,"불을 가지고 노는 소녀는 리스벳이 1년동안의 세계여행을 끝내고 스웨덴으로 돌아오는 것으로 시작한다. 귀국한지 얼마 되지 않았을때 리스벳은 자신의 후견인인 비여만과 블룸크비스트의 동료 두명의 유력한 살인용의자로 몰린다. 이것은 소비에트 연방의 옛 스파이이자 리스벳의 친부인 알렉산더 잘라첸코(Alexander Zalachenko)와 스웨덴 비밀경찰인 세포(Säpo)간부 몇몇과의 음모였다. 세포의 간부몇몇은 잘라첸코의 소비에트 연방에 대한 배신 이후 그를 지켜주는 임무를 받았었는데, 그들은 잘라첸코의 모든 악행위를 덮어주는 역할을 해왔었다.\n\n잘라첸코는 러시아 총정보국에서 높은 직위를 갖고있는 군인이었는데, 그가 소비에트 연방을 배신했을때 세포가 그것을 주의깊게 보고 그의 불법적 행위들을 다 덮어주며 그에게로 부터 소비에트 연방에 대한 정보를 조금씩 받았다.\n잘라첸코는 아들이 하나 있었는데, 그는 리스벳의 이복형제로, 로날드 니덜만(Ronald Neidermann)이다. 그는 잘라첸코의 성매매에 대해 조사하던 블룸크비스트의 두 동료를 살해하고, 리스벳에게 복수를 하려는 비여만을 살해했다. 그리고 잘라첸코의 모든 만행을 덮기위해 모든 일을 리스벳이 누명을 쓰게 만들었다.\n\n블룸크비스트는 리스벳을 도와주려 하지만, 리스벳은 블룸크비스트가 자신의 인생에 다시 연관되는 것을 원하지 않았다. 소설의 끝자락에선 블룸크비스트가 중상을 입은 리스벳을 구하기 위해 잘라첸코의 농장으로 가는데 그곳에서 도망가고 있는 니덜만과 리스벳으로 인해 얼굴과 다리를 도끼로 찍힌 후 죽어가고 있는 피투성이의 잘라첸코를 발견한다. 니덜만을 총으로 위협하며 포박하고 잘라첸코의 집에 의식을 잃은 리스벳을 본 블룸크비스트는 즉시 구급차를 부른다.\n\n라르손의 두 번째 소설에선 리스벳의 유년기를 묘사하였다. 리스벳은 천재적인 지능을 갖고있지만 사회에 적응하지 못하며 자신을 해하려는 사람에게는 무차별적으로 폭력적이라고 하였다. 이 모든 성격은 불우한 집안문제로 인한 결과였다. 잘라첸코는 반복적으로 리스벳의 어머니를 신체적과 정신적으로 폭력을 가했지만 스웨덴 정부는 잘라첸코의 존재가 리스벳의 엄마의 존재보다 더 중요하기 때문에 아무런 조치를 취하지 않았다. 잘라첸코는 리스벳의 쌍둥이 자매인 카밀라(Camilla)와 리스벳의 관계또한 망쳐놓았는데, 이는 카밀라는 잘라첸코를 상냥하고 점잖은 사람으로 기억하려 했기 때문이다. 예를 들어 잘라첸코가 방에서 문을 잠그고 리스벳의 어머니를 폭행한 후 나왔을때 카밀라는 잘라첸코에게 미움받지 않기위해 웃으며 안기기를 반복했기 때문이다.\n\n그러던 어느날, 리스벳이 12살 일때 잘라첸코는 리스벳의 어머니를 너무 심하게 폭행한 나머지 뇌손상을 입혔다. 그동안 아버지에 대한 복수를 꿈꾸던 리스벳은 휘발류를 모아왔는데, 그날 리스벳은 차에 탑승한 잘라첸코에게 휘발유를 들이부은 후 불붙은 성냥을 그에게 던졌다. 그로인해 잘라첸코의 외관은 심각하게 훼손되었고 만성질환을 앓게되었다. 후에 잘라첸코가 리스벳을 땅에 묻어버리기 전 지금까지 거울을 볼때마다 리스벳을 생각하며 증오한다고 했다.\n리스벳의 방화이후, 법적으로 정신이상자로 판결나고 웁살라에 위치한 소아정신병원에 수감입원되었다. 그곳에서 리스벳은 심리학자 닥터 피터 텔레보리안(Dr. Peter Teleborian)의 감시아래에 지내게 되었는데, 그 또한 잘라첸코와 관련이 있었으며 잘라첸코의 신분을 덮어두기 위해 모든것을 알고있는 리스벳을 평생 정신병원과 법원의 통제안에 가두어두려했다.\n리스벳이 웁살라의 정신병원에 입원했을 당시에는 텔레보리안의 감독아래 억제요법을 받았는데 이것은 아동성장애자인 텔레보리안의 성적욕구를 리스벳을 통해 풀기위함이었다. 텔레보리안은 리스벳을 법적으로 정신병자로 구분하였는데, 이는 자신들이 한 행동들을 묻어주기 위함이었다. 그리고 팔그렌의 뇌졸중 이후 자신들과 한패인 비여만을 리스벳의 후견인으로 임명함으로, 리스벳을 자신들의 감독안에 두기위해서였다."
1956,"스웨덴어는 인도유럽어족 게르만어파의 한 갈래인 북게르만어군에 속한다. 덴마크어와 함께 동스칸디나비아어 그룹에 속하며, 페로어, 아이슬란드어, 노르웨이어가 속하는 서스칸디나비아어 그룹과는 구분된다. 최근 연구에서는 노르웨이어를 동스칸디나비아어 그룹에 포함시켜 북게르만어군을 섬 스칸디나비아어와 본토 스칸디나비아어로 분류하는 경향도

In [36]:
special_char = "Þ"
train_df[train_df["context"].str.contains(special_char)]["context"]

,context
517,"노이는 이름 모를 아이슬란드 서쪽 외딴 어촌 마을에서 그의 할머니 리나 (Anna Friðriksdóttir)와 함께 사는 17살 소년이다. 아버지 키디 (Þröstur Leó Gunnarsson)는 알코올 중독자의 택시 운전수로 같은 마을에 살지만, 노이와 가까운 관계를 유지하진 못한다. 알비노인 그의 외모는 마을에 다른 사람들과 완전히 다르다. 그는 대부분의 시간을 황량한 마을이나 동네 서점을 어슬렁거리거나, 그만의 사적인 공간인 할머니 집의 비밀 공간에서 보낸다. 산으로 둘러 쌓여있고, 겨울에는 눈에 갇혀 산을 통과하는 도로가 막혀 오직 배만이 외부와의 접촉을 허락되는 마을에서 노이는 고통스러워한다. 지적으로 뛰어나지만, 그는 학업에는 완전 무관심하며 선생님과의 관계도 좋지 않다. 결석을 밥 먹듯이 하는 그는 주로 동네 주유소에 있는 슬롯머신을 조작하여 돈을 챙긴다. 노이에게 이 절망적인 마을에서의 미래는 불안할 뿐이다.\n\n노이의 상황은 마을에 새로 온 매력적인 주유소 점원 아이리스 (Elín Hansdóttir)를 만나며 변한다. 노이는 아이리스와의 서툰 연애를 시작한다. 어느날 밤 그들은 지역 자연사 박물관에 몰래 들어간다. 경비원에게 쫓겨 창고에 숨게 되었는데, 그 곳에서 그들은 불 빛이 나는 세계지도를 발견한다. 함께 지도를 보다 아이리스는 이 곳에서 벗어나자고 제안한다. 그 후 노이는 아이리스와 함께 마을과 아이슬란드를 떠나는 꿈을 갖게 된다. 한편, 생일을 맞아 할머니에게로부터 그는 열대 섬 사진이 들어있는 뷰 마스터 View-master를 받는다. 그는 자신 주변의 환경과 정반대인 열대 해안의 이미지에 얼어붙는다.\n\n그러던 어느 날, 수학 시간에 자신 대신 녹음기를 출석 시킨 노이는 선생을 화나게 만들었다. 선생은 교장에게 그를 퇴학시킬 것을 부탁하고, 노이는 결국 학교에서 쫓겨나게 된다. 사실을 알게 된 아버지는 처음엔 그에게 분노했지만, 미안해하며 곧 그를 마을 술집에 데려갔다. 하지만 술집에서 노이는 쫓겨나게 되고, 그날 밤 그는 아이리스를 만나러 그녀의 집에 갔지만 그녀의 아버지 오스카에게 발각된다. 오스카의 충고에도 불구하고, 아이리스는 노이가 밤새 그녀의 집에 머물 수 있도록 도와준다.\n\n한편, 그의 할머니는 마을 점쟁이 길피 (Kjartan Bjargmundsson)에게 노이의 운세를 봐주고, 그가 바른 길로 갈 수 있도록 도와달라고 부탁한다. 길피는 노이에게 죽음의 운명이 보인다고 말한다. 노이는 길피가 자신을 놀린다는 생각에 화를 내며 떠난다. 그 길로, 노이는 일을 관두고 할머니의 총을 꺼내 달아난다. 그는 은행을 털고자 했지만, 아무도 그를 심각하게 여기지 않았고 총을 빼앗으며 내쫓았다. 그는 다시 은행으로 돌아와 그의 계좌에서 돈을 꺼내 새로운 양복을 구입한다. 그는 새로운 옷을 입고 차를 훔쳐 아이리스에게 떠나자고 제안했다. 그러나 아이리스는 혼란스러워 하며 그와 떠나기를 망설였다. 결국 노이는 혼자 떠나게 되었지만, 곧 차는 눈에 갇혀 경찰에게 붙잡힌다.\n\n경찰서에서 돌아온 노이는 그만의 비밀 공간으로 내려간다. 그 순간 갑자기 땅이 흔들리며 빛이사라진다. 노이는 탈출하려 했지만 머리 위에 있는 문은 열리지 않고 꼼짝없이 갇히게 되었다. 결국 그는 외부의 도움으로 빠져 나오게 되었고, 그것이 산사태이었음을 알게 된다. 산사태로 그의 집은 무너졌고 아버지와 할머니의 목숨을 앗아갔다. 구조 쉼터에 있던 노이는 산사태로 아이리스와 길피를 포함한 그가 알고 있던 모든 사람이 죽었다는 것을 알게 된다. 시간이 지나고 그는 할머니 집터에서 그의 뷰 마스터를 발견하여 그것을 들여다 보았다. 영화는 노이가 바라보고 있던 열대 해변 사진이 천천히 진짜 해변의 모습으로 바뀌면서 끝난다."
1147,"토르그뉘르 로그마드(Þorgnýr lǫgmaðr)는 중세 초기에 살았던 로그마드들로, 토르그뉘르라는 이름을 가진 로그마드는 세 명 있다. 그 출전은 스노리 스투를루손의 헤임스크링글라, 그리고 그보다는 덜 알려져 있는 스비아인의 대전사 스튀르뵤른의 사트르, 바보 흐로이의 사트르가 있다. 세 명의 토르그뉘르는 모두 티운달란드의 로그마드였으며 스웨덴 왕국을 섬겼다. 이 이야기들이 모두 역사적 사실이라면, 세 명의 토르그뉘르 로그마드는 3대 부자 관계다.\n\n가장 유명한 토르그뉘르는 헤임스크링글라에 그 행적이 전하는 인물로, 스웨덴의 올라프 쇤스키와 노르웨이의 올라프 헬가의 치세에 살았다. 스노리에 따르면, 티운달란드에 토르그뉘르라는 이름의 로그마드가 살았는데, 그 토르그뉘르의 아버지는 토르그뉘르의 아들 토르그뉘르(Þorgnýr Þorgnýrson)이었다. 토르그뉘르의 가계는 오랫동안 티운달란드의 로그마드로서 많은 왕들을 섬겼다. 토르그뉘르는 늙었을 때 스웨덴에서 가장 지혜로운 사람으로 여겨졌으며, 라근발드 울프손의 양부였다.\n\n당시 스웨덴과 노르웨이의 두 명의 올라프 왕이 서로 전쟁을 벌였다. 1018년, 뵤른 스탈라레와 햘티 스케기아손 등 노르웨이 대표단이 웁살라 팅그에 와서 스웨덴 왕녀 잉게게르드 올로프스도테르를 노르웨이 왕비로 내놓으라고 요구했다. 라근발드 울프손도 이를 거들었다. 올로프 쇤스키는 노하여 라근발드를 추방하려 했다. 그러나 라근발드의 양부 토르그뉘르가 스웨덴에서 가장 존경받는 원로로서 라근발드의 역성을 들었다. 토르그뉘르가 조리있게 일장연설을 하자 그 자리의 전원이 박수를 치며 환호했다. 결국 올로프 쇤스키는 딸을 내주겠다고 하며 노르웨이와 평화를 맺었으나, 나중에 그 약속을 어겼다.\n\n올라프 뵤른손의 아들이며 에이리크 인 시그르셀리의 조카인 스튀르뵤른 스테르키에 관한 단편인 스비아인의 대전사 스튀르뵤른의 사트르에는 다른 토르그뉘르 로그마드가 나온다. 이 토르그뉘르 로그마드는 헤임스크링글라의 토르그뉘르 로그마드의 아버지인 것 같다. 이 토르그뉘르는 매우 늙어서 눈도 거의 멀었고, 에이리크와 스튀르뵤른 사이에 벌어진 전쟁에 참여하지 못했다. 하지만 소떼와 말떼가 데인인 군대 쪽을 향해 달려가도록 하여 싸움에 도움을 주었다. 이 가축 떼는 스튀르뵤른 측에 큰 피해를 입혔다."
1547,"《암탉 토리르의 사가》(Hænsa-Þóris saga|헨사토리스 사가)는 아이슬란드 사가 작품 중 하나이다. \n\n주인공 헨사토리르(Hænsa Þóris)는 장사로 부유해져 땅을 사게 된 사람으로 주위에 인망이 없었다. 이야기 내내 토리르는 보다 제대로 된 배경을 가지고 있는 이웃사람들과 부정적으로 비교된다. 이웃들이 겨울나기에 필요한 짚단을 팔라고 요구하자 토리르는 이를 거부한다. 그러거나 말거나 이웃을이 멋대로 짚단을 가져가 버리자 토리르는 이웃들의 집에 불을 질러 그들을 산채로 태워 죽인다. 이에 대한 복수가 뒤따라 토리르도 살해당해 목이 잘리는 것으로 이야기가 끝난다.\n\n《암탉 토리르의 사가》는 중세 아이슬란드 문화의 여러 면을 보여준다. 예컨대 나그네를 환대해야 하고 이웃들에게 관대해야 하며 사법적 판단을 위해서는 추장의 지지를 얻어야 하는 것 등이 그러하다. 또다른 학설로는 당시 노르웨이 국왕 마그누스 6세가 법률을 개정한 것과 이 작품이 유관하다고 한다. 새로운 법에 따르면 농부는 이웃들이 절박한 필요로 한다면 그들에게 건초를 팔아야만 하는 의무가 있었다. 건초 판매를 거부하면 벌금을 물었고, 무력으로 저항한다면 이웃들이 그를 공격해도 범죄로 취급되지 않았다. 이는 아이슬란드 전통 법제사에 유례가 없던 새로운 법으로서, 별로 인기가 없었던 것으로 보인다. 이 학설에 따르면 《암탉 토리르의 사가》는 새 법률을 홍보하기 위한 선전물로서 쓰여진 것이다."
3496,"토라 보르가르효르트(Þóra Borgarhjǫrtr)는 노르드 신화에 나오는 인물이다. 라그나르 로드브로크의 두 번째 아내이며 예탈란드 야를 헤라우드의 딸이다.\n\n토라는 베스테르예탈란드의 내실에 살았다. 아버지 헤라우드는 토라에게 작은 린트부름을 애완동물로 주었는데 린트부름이 너무 커져서 내실을 통째로 휘감을 지경이 되었고 토라는 그 안에 갇혀 버렸다. 헤라우드는 누구든 린트부름을 퇴치하는 남자와 토라를 결혼시키겠다고 약속했다.\n\n첫 번째 부인 라드게르타와 이혼한 뒤였던 라그나르는 

In [37]:
special_char = "क"
train_df[train_df["context"].str.contains(special_char)]["context"]

,context
217,"노예제도가 폐지 된 후, 유럽 국가의 많은 식민지와 미국에서 노동력이 부족했다. 영국의 식민지였던 인도 대륙의 빈민층과 아편전쟁 후, 광둥성, 푸젠성 두 성을 중심으로 산터우, 샤먼, 마카오 등지에서 저임금의 노동력을 가진 쿨리가 세계 각지에 보내졌다. 처음에는 ‘인도 노동자’(क़ुली)를 가리키는 호칭이었지만 나중에 중국인 노동자들에게 “쿨리”(苦力)라는 한자가 음역되었다. 미국은 대륙횡단철도 건설 노동자 등으로 고용되었고, 중국에서 캘리포니아로 10만명 이상이 보내졌다. 호주와 말레이시아, 마다가스카르 등에도 각각 10만명 정도가 이주했다고 한다. 공식적인 해외 유입은 베이징 조약 체결 이후가 되었지만, 그 이전에도 사실상 중국에서 쿨리 수출은 이루어지고 있었다. 그 배경에는 중국의 인구 증가, 태평천국 이후의 사회적, 시대적 불안이 있었다. 쿨리는 열악한 환경에서 생활했기 때문에 항해 중 또는 작업 중에 사망하는 경우가 많았고, 현지에서도 최하층의 생활을 강요당했지만, 자손들 중에는 그러한 환경을 벗어나 사회 신분이 상승된 이도 적지 않았다. 또한 1937년 중일전쟁 때에는 만주와 일본의 점령지에서도 쿨리가 사역을 했지만, 중국이 공산화되면서 사실상 쇄국 정책이 채택되고, 쿨리 무역은 종결되었다."


In [38]:
special_char = "ㄱ"
train_df[train_df["context"].str.contains(special_char)]["context"]

,context
661,"정무공 오정방(1588∼1634), 천파공 오상, 충정공 오두인(1624∼1689) 등 해주 오씨의 이름난 학자들을 배출한 유서깊은 곳이다. 특히 충정공은 이 집에서 성장하였으며 나라에 큰 공헌을 한 일이 있어 우암 송시열이 글을 써서 보내오기도 하였다. 중종 5년(1510) 덕봉리에 처음 세웠으나 효종 1년(1650)에 지금의 위치로 옮겨 지었다. 현재 문간채, 안채 겸 사랑채, 사당이 남아있다.\n\n이 집은 일반적으로 안채와 사랑채가 떨어져 있는 것과는 달리 연속하여 하나의 건물에 구성하였는데, 특히 ㄱ자형 몸채에는 일자형으로 부분을 길게 연장하고 중간에 담을 두어 안팎으로 나눈 수법은 이 집의 특징이다.\n\n안채 부분은 대청과 건넌방, 안방, 부엌 등을 두었고 사랑 부분은 툇마루를 두고 기단을 높게 하여 개방성을 두드러지게 나타냄으로써 실제 규모에 비해 장중하게 느껴진다. 사당은 사랑채 뒤쪽의 한쪽에 별도로 담장을 둘러 일곽을 이룬다. \n\n가옥의 배치 및 구성과 함께 건물의 구석구석에서 느낄 수 있는 목수의 안목과 솜씨가 높이 평가되는 주택이다.\n\n이 건물은 공간의 다양성을 살리면서 남녀의 생활 공간을 적절히 구분하였고, 전체적으로 소박하게 꾸민 편이다. 조선 후기 사대부 가옥의 구조를 잘 보여주고 있으나 안채와 사랑채를 엄격히 구분하였던 17세기 사대부 가옥의 일반적인 경향과는 달리 안채와 사랑채를 연결한 독특한 구조를 보인다. 조선 후기 주택사 연구에서 중요한 자료로 꼽힌다."
2247,"안채와 사랑채, 행랑채와 곳간채, 문간채로 구성되어 있으며 1938년에 지은 집이다.\n\n안채는 앞면 5칸·옆면 3칸이며, 지붕은 옆면에서 볼 때 여덟 팔(八)자 모양을 한 팔작지붕이다. 천장 위에는 모두 다락을 두었고 앞면과 뒷면 창은 빗살창으로 꾸몄다.\n\n사랑채는 안채 앞쪽에 배치되어 있는 건물로 지붕은 팔작지붕이다. ‘영춘헌’이라는 현판이 걸려 있으며, 안채와 같이 천장 위에 다락을 만들었고 ㄱ자형으로 뻗친 대청누각이 있다.\n\n곳간채는 안채와 사랑채 사이에 남북으로 뻗어 있다. 앞면 4칸·옆면 1칸 규모로 지붕은 옆면에서 볼 때 사람 인(人)자 모양을 한 맞배지붕이다.\n\n행랑채는 곳간채 낮은 편에 서 있는데 앞면 4칸·옆면 1칸 크기에 지붕은 맞배지붕으로 꾸몄다.\n\n안채와 사랑채를 통하는 안문은 팔작지붕이며 그 옆에 조그마한 문이 있다. 문의 윗부분을 자연스럽게 타원형으로 짠 것이 인상적이다.\n\n문간채는 앞면 3칸·옆면 1칸으로 대문을 중심으로 좌우에 하나씩 방이 있으며 대청 뒤쪽에서 남으로 뻗은 누각 형태의 마루가 시원한 느낌을 주고 있다."
3062,"나지막한 언덕에 자리하고 있는 옛집으로, 조선 숙종(재위 1674∼1720) 때 이조판서를 지낸 이자가 고향으로 돌아와 지은 것이라 한다. 지은 연대는 정확히 알지 못하며, 90여 년 전에 안채를 크게 수리하였다고 한다.\n\n안채와 사랑채가 ㅁ자형으로 배치되었고, 동남향을 하고 있다. 1m 정도의 비교적 높은 기단 위에 지었으며, 여덟 팔(八)자 모양의 화려한 팔작지붕집이다. 안채는 영동지방에서 흔히 볼 수 있는 겹집형에, 영서·중부지방에서 일반적인 대청과 건넌방이 ㄱ자형으로 접하여 있다. 대청에는 모두 문을 달았고, 건넌방과 대청·안방부분에도 툇마루를 달아 외부와 연결을 주었다.\n\n부엌의 앞면에서 광·외양간과 대문·사랑채가 ㄷ자형으로 이어졌다. 사랑채는 큰사랑·작은사랑·서함·대청·머슴방으로 구성되었고, 특히 외양간이 대문 오른쪽에서 돌출된 것이 특이하다. 사랑채의 대청에도 문을 달았고, 앞뒷면에 툇마루를 두어 시원한 공간구성을 하고 있다.\n\n사랑채의 서쪽에는 별채인 수고당이 있으며, 대청·작은마루·다락·서고로 구성되어 있다. 수고당에는 외재 이단하 내외분의 옷(국가민속문화재 제4호)이 있어 조선시대 의복연구에 중요한 자료가 되고 있으며, 이 외에 각종 시문초고·교지 및 전적과 간찰·서화가 보존되어 있다."
3372,"2020년 3월 27일 전북 전주시에 사는 40대 ㄱ씨가 군산의 한 병원에 진료차 방문했고 그 병원에선 ㄱ씨의 해외여행 이력을 들어 코로나 검사를 받아야 진료를 받을 수 있다 안내하며 군산보건소 선별진료소로 보냈고 ㄱ씨가 보건소 직원에게 ""전주에 사는데 군산에서 검사 받아도 되느냐?"" 묻자 직원이 괜찮다고 답해 1시간 가량 기다렸으나 직원이 자기 주소지에서 검사받아야 비용면제하도록 방침 바뀌었다며 전주에서 검사받기 권하자 ㄱ씨가 ""왜 미리 안내해 주지 않았느냐"" 며 언성 높이고 화를 내었고 주변의 직원들이 ""여기 시장님이 계시니 조용히 해달라""고 시장을 의식하는 말만 했다하며 ㄱ씨는 자신의 승용차에 타서 집에 돌아가버리려는데 보건소 한 직원이 ㄱ씨 차를 막아서며 오해 풀자고 이야기하던 와중 당시 보건소에 있던 강 시장이 떠난다는 소리를 듣자 그 직원이 ㄱ씨와 이야기를 끊고 시장 차량으로 향해버리자 ㄱ씨가 다시 언성을 높이며 ""시장이 간다고 사람을 세워두느냐? 난 시장 낮짝도 모른다. 시장은 사람이고 시민은 사람이 아니냐?"" 고 이야기하자 강 시장이 차에서 내리며 ""내가 시장이다 XX야. 어린놈의 XX. 뚫린 입이라고 싸가지 없게. 저런 것은 집어넣어 버려야 해"" 이런 식으로 욕을 했다는 것이다. 그리고 주변 직원들이 ㄱ씨를 움직이지 못하게 막고 그 사이에 시장이 차를 타고 빠져나갔다는 것인데 이에 대해 ㄱ씨가 한 SNS에 ""시민을 생각하고 시민의 소리를 듣고 시정활동 해야하는 사람이 시민에게 욕설과 폭언 하는 게 말이 되느냐? 지나가던 시민들도 그 상황을 봤을 것이다. 저 어린 놈 아니다. 고등학생 자녀가 있고 마흔이 넘은 나이다"" 라고 글을 올렸고 그걸 본 사람들이 댓글로 시장을 비난하자 군산시 직원들이 ㄱ씨에게 연락해 사과했고 나중에 강 시장의 사과전화를 받고서야 마음을 풀었다 한다.\n\n강 시장은 언론들과의 통화에서 코로나 사태 때문에 보건소 직원들이 고생하는데 ㄱ씨가 고함지르는 거 보고 감정 추스르지 못하고 실수를 범했다며 ㄱ씨를 만나 오해를 풀었다며 이런일이 일어나 송구하다고 밝혔다. 조선일보 한겨레"


In [39]:
special_char = "ㅎ"
train_df[train_df["context"].str.contains(special_char)]["context"]

,context
367,"2010년 4월 30일, 라리사는 29세 센터백과 협상에 성공하여 이 해 여름에 녹색 군단과의 계약이 만료되면 자유이적으로 팀에 합류하게 될 것이라 발표하였다. 프랑스에서 황소같은 플레이스타일로 '황소' (Le Taureau) 라는 별칭을 얻은 그는 릴과 생테티엔 선수로 활약해 리그 1에서 성공한 그리스 선수이나, 알랭 페랭 감독에 의해 스타드 조프리 귀샤르 연고 팀의 명단에서 밀려났다. 전 올랭피크 리옹 감독이 경질된 후, 타블라리디스는 크리스토프 갈티에르 감독 하에 인상을 줄 기회를 잡았으나, 갈티에르 임기의 첫 경기인 마르세유전에서 10분만에 퇴장당하면서 녹색 군단으로써의 ㅎ뢀약이 사실상 끝나버렸다. 나중에 합류할 구단의 공식 웹사이트에서, 그리스 인터넷 언론에 그는 ""저는 제 경력을 해외에서 이어나가는 것을 우선순위로 잡았었습니다. 저는 그리스에 복귀할 여부를 4월까지 결정했으며, 9년만의 복귀는 제가 가족을 거느리는 것으로 인해 만반의 준비를 필요로 합니다. 저는 그리스의 서너개 구단의 추천을 받았으나, 심사숙고 끝에 저는 라리사에 입단하기로 결정했습니다."" 타블라리디스는 전에 1월에 그리스 클럽의 진심한 제의를 받았으나, 차남의 출생을 지켜보기 위해 거절했었다. 그는 지난 시즌을 8위로 마친 수페르리가 엘라다 클럽과 4년 계약을 맺었다.\n\n그러나, 2011년 8월 9일, 소속팀이 풋볼 리그로 강등당함에 따라 계약을 해지시켰다."
3684,"모아주기는 한글 글자를 초성, 중성, 종성 순서에 상관없이 오토마타가 지능적으로 합성해 주는 기능이다. 모아치기는 문자적으로 키보드 입력을 짧은 시간의 차이 안에 동시에 입력하는 경우를 뜻하며, '동시'에 입력되는 키를 같은 음절로 결합해주는 기능이다. 모아치기의 반대는 이어치기이다. 모아치기의 이론은 있었지만, 실제 발명은 안마태가 최초라고 주장된다. 그러나 속기 자판에서는 이미 초/중/종성의 모아치기를 하고 있었으므로 이 주장은 논란의 여지가 있으며, 일반 키보드에 속기 자판의 장점을 활용하였다고 보아야 한다. 모아치기와 이에 대한 세부적인 용어는 아직 공식적으로 확실하게 정해지지 않았다. \n\n# 이어치기(순차입력): 글자를 음소 단위로 하나씩 쳐서 입력하는 방식을 말한다. (예: 한 = ㅎ + ㅏ + ㄴ받침) 풀어치기라고도 하는데, 그렇게 말하면 풀어쓰기와 혼동할 여지가 있다. 그래서 날개셋 입력기의 경우는 '이어치기'라는 용어를 사용한다.\n# 모아치기: 함께 입력된 음절의 모든 자모 글쇠를 같은 음절로 모아주는 기능이다. 한 음절을 구성하는 모든 글쇠를 한 타로 동시에 쳐서 입력하는 것을 말한다. 안마태 연구소에서는 이에 대해 '동시치기'라고 정의했다 (예: 한 = ㅎ/ㅏ/ㄴ받침 한번에) 안마태 소리글판이나 속기 자판 등의 모아치기용 자판만 가능.\n# 부분 모아치기(나눠치기): 함께 입력된 음절의 일부 자모 글쇠를 같은 음절로 모아주는 기능이다. 즉, 한 음절을 두 번 이상으로 나누어서 입력한다. 안마태 연구소에서는 한 음절을 두 번에 나누어 넣는 것에 대해서만 '모아치기'라고 정의했다. (예: 한 = ㅎ/ㅏ + ㄴ받침) 세벌식 자판만 가능.\n# 모아주기: 시차와 관계 없이 입력된 글쇠를 같은 음절로 모아주는 기능이다. 손으로 모아서 친다는 의미가 아닌 오토마타가 글자를 모아준다는 의미로 많이 쓴다. 따라서 이 경우, 모아치기는 잘못된 표현임이 분명하다. 모아주기가 뜻에 맞을 것이다."


In [40]:
special_char = "ㄹ"
train_df[train_df["context"].str.contains(special_char)]["context"]

,context
2656,"다른 언어로 노래하기 위한 목적으로 노랫말을 번역하는 것을 노래 번역으로 부르기도 한다. 노래 번역은 음성이 담긴 음악이 대부분으로 구성된 노래를 다루기에 시 번역과 밀접한 연관성을 가지며 압운과 각운 등 여러가지 운율도 고려되어야 한다. 19세기 이래로 산문과 자유 시 구조가 일부 예술적 음악 장르에서 다루어 졌지만 대중 음악에서는 운율(스탠자)을 유지함에 있어서는 상당히 보수적인 입장을 취했다. 시를 노래를 위해 번역한 가장 기초적인 예가 찬송가이다. 독일의 합창성가를 영어로 옮긴 주번역자가 캐서린 윙크워스이다. \n\n노랫말의 번역은 대개 시구 번역보다 훨씬 제약이 많다. 이는 형태에 있어 거의 혹은 전혀 자유가 없어 다양한 번역을 창조해내기가 불가능하고 시구 구조에서도 다른 여지를 찾기 어려운 탓이다. 누군가는 각운을 번역 과정에서 생략하거나 수정해낼 수 있겠지만 각 음절의 번역이 특정한 음표를 나타내는 음악적 기조를 따라가려면 번역가는 엄청난 노력을 들여야 한다. 산문처럼 길게 늘어뜨린 노랫말의 경우 시구처럼 짧은 노랫말보다는 어려움이 덜할 수 있는 이유가 음운을 맞추기 위해 음절을 이곳 저곳에서 떼거나 분할해 적용할 수 있기 때문이다. 그런 사실을 인정하더라도 과정상 엄밀히 말해 시구 번역에 해당하는 노랫말 번역은 가능한한 노래의 음절을 맞춰 진행해야하는 노래의 특성상 제약이 많을 수 밖에 없다.\n\n다른 고려사항으로는 노랫말 번역시 단어와 글귀를 반복하거나 나머지를 대체하는 행위 혹은 구두점, 자음의 사용에 있어 고음에 맞춰 사용하기 등이다. 이외에도 리듬에 맞춰서 사용했을 시 도착어에서 부를 때 훨씬 부드럽고 자연스럽게 적용될 수 있을 자음을 택하는 것도 이러한 작업에 해당한다. 노랫말 번역은 상당히 그리고 완벽하게 원래판과 달라질 수 있다. 예를 들어 영어로 된 노래를 한국어로 번역할 때 ㄹ이나 ㄴ 처럼 콧소리를 유발하는 음절이 계속해서 나오거나 발음이 꼬일 경우에는 다른 유사한 의미의 단어를 취하거나 대체해 노랫말이 간결하게 전달될 수 있도록 돕는 일도 포함될 수 있다.\n\n노랫말의 번역은 노래를 하거나 좀 더 혹은 덜 문학적 형태를 띨 수 있겠지만 청중이나 가수, 지휘자를 위한 배려의 목적에서 고려되기도 한다. 잘 모르는 언어로 설명될 때가 그렇다. 가장 흔히는 번역이 자막이나 오페라의 대화가 번역되어 영사기로 보이는 것에 해당한다. 콘서트 프로그램 진행에도 적용되며 상업 오디오 CD-ROM에도 이런 경우가 발견된다. 더욱이 전문적이거나 아마추어 가수들이 보통 잘 모르는 언어로 노래할 때 의미를 이해할 수 있도록 번역하여 부르거나 번역을 이용하는 모습을 볼 수 있다."
3690,"다른 언어로 노래하기 위한 목적으로 노랫말을 번역하는 것을 노래 번역으로 부르기도 한다. 노래 번역은 음성이 담긴 음악이 대부분으로 구성된 노래를 다루기에 시 번역과 밀접한 연관성을 가지며 압운과 각운 등 여러가지 운율도 고려되어야 한다. 19세기 이래로 산문과 자유 시 구조가 일부 예술적 음악 장르에서 다루어 졌지만 대중 음악에서는 운율(스탠자)을 유지함에 있어서는 상당히 보수적인 입장을 취했다. 시를 노래를 위해 번역한 가장 기초적인 예가 찬송가이다. 독일의 합창성가를 영어로 옮긴 주번역자가 캐서린 윙크워스이다. \n\n노랫말의 번역은 대개 시구 번역보다 훨씬 제약이 많다. 이는 형태에 있어 거의 혹은 전혀 자유가 없어 다양한 번역을 창조해내기가 불가능하고 시구 구조에서도 다른 여지를 찾기 어려운 탓이다. 누군가는 각운을 번역 과정에서 생략하거나 수정해낼 수 있겠지만 각 음절의 번역이 특정한 음표를 나타내는 음악적 기조를 따라가려면 번역가는 엄청난 노력을 들여야 한다. 산문처럼 길게 늘어뜨린 노랫말의 경우 시구처럼 짧은 노랫말보다는 어려움이 덜할 수 있는 이유가 음운을 맞추기 위해 음절을 이곳 저곳에서 떼거나 분할해 적용할 수 있기 때문이다. 그런 사실을 인정하더라도 과정상 엄밀히 말해 시구 번역에 해당하는 노랫말 번역은 가능한한 노래의 음절을 맞춰 진행해야하는 노래의 특성상 제약이 많을 수 밖에 없다.\n\n다른 고려사항으로는 노랫말 번역시 단어와 글귀를 반복하거나 나머지를 대체하는 행위 혹은 구두점, 자음의 사용에 있어 고음에 맞춰 사용하기 등이다. 이외에도 리듬에 맞춰서 사용했을 시 도착어에서 부를 때 훨씬 부드럽고 자연스럽게 적용될 수 있을 자음을 택하는 것도 이러한 작업에 해당한다. 노랫말 번역은 상당히 그리고 완벽하게 원래판과 달라질 수 있다. 예를 들어 영어로 된 노래를 한국어로 번역할 때 ㄹ이나 ㄴ 처럼 콧소리를 유발하는 음절이 계속해서 나오거나 발음이 꼬일 경우에는 다른 유사한 의미의 단어를 취하거나 대체해 노랫말이 간결하게 전달될 수 있도록 돕는 일도 포함될 수 있다.\n\n노랫말의 번역은 노래를 하거나 좀 더 혹은 덜 문학적 형태를 띨 수 있겠지만 청중이나 가수, 지휘자를 위한 배려의 목적에서 고려되기도 한다. 잘 모르는 언어로 설명될 때가 그렇다. 가장 흔히는 번역이 자막이나 오페라의 대화가 번역되어 영사기로 보이는 것에 해당한다. 콘서트 프로그램 진행에도 적용되며 상업 오디오 CD-ROM에도 이런 경우가 발견된다. 더욱이 전문적이거나 아마추어 가수들이 보통 잘 모르는 언어로 노래할 때 의미를 이해할 수 있도록 번역하여 부르거나 번역을 이용하는 모습을 볼 수 있다."


### 5.question과 answer에도 다른 언어가 존재할까?

In [54]:
answers = train_df["answer_text"]
print(len(answers))
answers.head()

3952


,answer_text
0,하원
1,《경영의 실제》
2,백성
3,중국
4,4개


In [67]:
# 특수 문자를 찾는 함수 정의
def find_special_characters(text, pattern):
    # 정규 표현식을 사용하여 특수 문자 찾기
    return re.findall(pattern, text)

# context에서 모든 특수 문자를 찾고 중복 제거
all_special_chars = set()
pattern = r'[^a-zA-Z0-9가-힣\s]' # 알파벳, 숫자, 한글, 공백 제외
#pattern = r'[^a-zA-Z0-9가-힣\s\u4e00-\u9fff]' # 알파벳, 숫자, 한글, 한자 공백 제외
#pattern = r'[^a-zA-Z0-9가-힣\s\u4e00-\u9FFF\u3040-\u309F\u30A0-\u30FF\u0600-\u06FF]' # 알파벳, 숫자, 한글, 한자, 일본어, 아랍어 공백 제외
for idx, context in enumerate(train_df['answer_text']):
    special_chars = find_special_characters(context, pattern)
    all_special_chars.update(special_chars)

# 결과 출력
print(f"찾은 특수 문자 종류: {all_special_chars}")
print(f"특수 문자의 개수: {len(all_special_chars)}")

찾은 특수 문자 종류: {'月', '之', '流', '宋', '片', 'な', '陽', '祚', '意', '忠', '資', '勳', '駅', "'", '魯', 'に', '蔣', '卵', '務', '」', '愛', '弼', '°', 'よ', '\xad', '>', '堂', '成', '一', '呂', '祠', '興', '儼', '＞', '趙', '部', '〈', '寺', '北', '城', '京', '扶', '蓮', '“', '重', '王', '式', '槪', '鉱', '+', '淵', '＜', '超', '有', '『', '河', '信', '鄭', '淸', '念', '餘', 'ㅋ', 'の', 'し', '牲', '海', '次', '/', '≫', '匡', '二', '藏', '張', ',', '康', '琿', 'ら', '汪', '‘', ')', '〉', '”', '毅', '學', '劉', '》', '心', '.', '原', '總', '雲', '限', '狼', '循', '性', '監', '"', '田', '理', '·', '身', '割', 'ä', '害', '祭', '大', '輪', '軍', '分', '麻', '塡', '甚', '陳', '華', '!', '臺', '奎', '縣', '《', '辣', '坂', '釋', '日', '郵', '∧', ':', '≪', '格', '金', '壇', '受', '夔', '塔', '遷', 'う', '<', '優', '’', '・', '太', '(', '陵', '』', '池', '「', '刀', '-', '媒', '前', '識', '肥', '冬', '公', 'い', '南', '~', '%', '孟', '文', '平'}
특수 문자의 개수: 160


In [70]:
# \xad가 어떤식으로 포함되어 있는지 확인
train_df[train_df["answer_text"].str.contains("\xad")]

,title,context,question,answer_start,answer_text,document_id,languages
544,강중인,"경상북도 출신으로 1935년에 경성제국대학 법문학부 법학과를 졸업했다. 1938년에 경성지방법원 및 경성지방법원검사국 사법관시보가 된 것을 시작으로 법조계에서 근무를 시작했다.\n\n1939년에는 경성지방법원 검사대리가 되었고, 1940년에 예비검사를 거쳐 그해부터 1943년까지 조선총독부 검사로 근무했다. 강중인은 경제담당 검사로 일하면서 태평양 전쟁 중 시국 연설과 기고로 사회 활동을 병행했다. \n\n1942년에 조선임전보국단 주최로 경성중앙방송국에서 저명 인사의 시국연설을 방송했을 때, 〈필승사상전(必勝思想戰)〉이라는 제목으로 연설한 일이 있다.\n\n《삼천리》에 〈최근의 반도의 경제사범 ­국민의 신경제 윤리의 파악을 위하여­〉라는 글을 기고하기도 했다. 이 글에서는 경제사범을 ""미증유의 국난""을 돌파해 ""대동아를 건설하는"" 국가적으로 중대한 순간에 사리사욕에만 눈이 먼 집단으로 묘사하고, 이들은 국민의 일대수치이므로 엄벌주의로 대처할 것이라 경고했다. 또한 관민이 일체가 되어 ""일사보국의 성을 다함""으로써 성업을 어서 완수해야 한다는 주장을 폈다.\n\n1943년에 대전지방법원 검사로 자리를 옮겨 일본 제국이 태평양 전쟁에서 패망할 때까지 근무했다. \n\n미군정 하의 서울에서 변호사를 개업하면서 좌익 운동에 뛰어들었다. 좌익 법조인 모임인 법학자동맹과 남조선로동당에 가입해 활동했다. 강중인이 남로당에 가입하게 된 동기는 후에 법정에서 다음과 같이 밝힌 바 있다. \n\n일정(日政) 당시에는 내가 내 한 몸을 구하기에 여력이 없었습니다. 8·15 해방을 맞이하자 피고는 이때야 깨달은 바가 있었습니다. 이는 다름이 아닙니다. 내가 어찌하면 국가와 민족을 위하여 일할 수 있을까 함이었습니다. 해외에서 국가와 민족을 위하여 투쟁하던 위대한 애국자들이 해방된 조국을 찾아 들어옴을 볼 때 무어라고 표현할 수 없는 느낌이 있었습니다. 그 후 피고는 남로당에 가입을 작정하였던 것이며, 법맹에도 초창기부터 가입하였습니다.\n\n1946년 민주주의민족전선 토지문제연구위원에 임명되었고, 미군정 과도정부의 사법부 총무국장도 지냈다. 같은 해 7월에 정판사 위조지폐 사건이 발생하여 남로당과 미군정이 정면 충돌을 일으켰을 때 위폐 사건 피고인들의 변호를 맡았다. 남로당은 곧 불법화되었고, 강중인은 1949년에 발생한 법조프락치 사건으로 구속되었다. 1심 재판에서 징역 3년형을 언도받고 항소 중에 한국 전쟁이 발발하여 조선민주주의인민공화국으로 갔다.\n\n2008년 민족문제연구소가 발표한 민족문제연구소의 친일인명사전 수록예정자 명단 중 사법 부문에 포함되었다.",강중인이 경제사범을 비판하며 국민의 일대수치라 표현한 글은?,324,최근의 반도의 경제사범 ­국민의 신경제 윤리의 파악을 위하여­,15575,"Korean, English, Chinese"


In [71]:
# 해당 언어거 어떤식으로 포함되어 있는지 확인
train_df[train_df["answer_text"].str.contains("な")]

,title,context,question,answer_start,answer_text,document_id,languages
1466,산리즈카 투쟁,"나리타공항 건설에서 화근이 된 일본 정부의 정책과 그로 인해 발생한 산리즈카 투쟁은 양측에 모두 비참하기만 한 결과를 가져왔다. 이후 공공사업을 둘러싸고 분쟁이 일어나는 현장에서는 “합의형성의 노력을 하지 않은 채 힘에 의지해 사업을 진행하면 힘에 의한 저항을 낳는다”는 자숙을담아 “나리타처럼 되지 말자(成田のようにならないようにしよう )”가 표어가 되었다. \n\n내륙에 공항을 건설하면 토지취득 및 소음 문제가 현저하게 발생할 수밖에 없는데, 나리타에서는 산리즈카 투쟁까지 더해져 막대한 손실을 초래했다. 이 교훈으로 이후 일본은 해상 및 원격지에 공항을 건설하는 경향이 커졌고, 이는 일본 공항의 편리성 저하와 비용 증가로 이어졌다.group=주해|그러나 해상에 건설된 간사이 국제공항에도 중핵파가 공항건설공사의 견학선에 방화하는 테러사건(요코하마요트 소형여객선 폭파사건)을 일으킨 바 있다. group=주해|간사이공항 인공섬 건설 총 공사비가 1조 엔임을 처음 들은 세계 공항관계자는 이 막대한 건설비에 통역의 오역이 아닌지 반문하기도 했다. 또한 토지수용과 행정대집행도 신중하게 실시되도록 되었으니, 산리즈카 투쟁은 일본 공공사업 전반에 엄청난 영향을 미친 것이다 \n\n나리타 하늘과 대지의 역사관은 국가공무원 종합직 초임자와 나리타공항회사 신입사원 연수에 포함되어 있다. 제2의 나리타를 만들지 않도록 산리즈카 투쟁의 교훈을 살리기 위한 노력이라 한다 \n\n한편 개발에 반대하는 주민 측에서도 실력투쟁에서 주민투표 등으로 투쟁방식을 바꾸는 전환점이 되었다 특히 장기간에 걸친 투쟁을 거쳐 이루어진 나리타 공항 문제 심포지엄・나리타 공항 문제 원탁회의는 이후 공공사업 실시 시의 모델 케이스가 되어, 나가라강 하구둑 문제도 원탁회의 방식이 채택되었고, 얀바댐 반대운동을 하던 주민들이 공부하러 나리타 심포지엄을 방문하는 등 일본 전국 각지에서 행해지던 주민운동의 참고사례가 되었다. \n\n관제탑 점거사건 등은 일본 국외에도 크게 보도되어 산리즈카 투쟁의 모습은 영화나 음악 등 예술작품의 테마로 사용되었고, 공공사업의 실패사례로서 큰 주목을 받았다. 예컨대 서독에서는 나리타와 같은 시기에 계획된 뮌헨의 공항건설에 있어서 나리타 공항 문제에 대한 철저한 연구분석을 선행했다. 1969년 공항건설을 결정한 바이에른주 정부는 20년간 259회에 이르는 공청회를 개최하여 반대파를 설득했고, 공항계획이 일부 축소되기는 했지만 최종적으로 착공 5년 후인 1992년 5월 프란츠 요제프 슈트라우스 공항이 개항되어 오늘날 유럽 공항의 일각을 이루고 있다.",공공사업과 관련한 갈등이 유발되는 현장에서 사용되는 문구는 무엇인가?,158,“나리타처럼 되지 말자(成田のようにならないようにしよう )”,38632,"Korean, English, Japanese, Chinese"


In [73]:
# 해당 언어거 어떤식으로 포함되어 있는지 확인
train_df[train_df["context"].str.contains("الكاشغري")]

,title,context,question,answer_start,answer_text,document_id,languages
2869,마흐무드 알카슈가리,"마흐무드 알카슈가리는, 그의 니스바 알카슈가리(الكاشغري)에서 알 수 있듯, 카슈가르에서 태어난 것으로 보인다. 태어난 시기는 1029년 경 또는 1038년 경이라는 것이 일반적인 설명이나, 990년으로 추측하는 학자도 있다. 그의 가문은 카라한 칸국 통치왕가에 속했던 것으로 보인다. 그러나 막상 마흐무드 자신은 이식쿨 인근의 바르스간에 애착을 보였는데, 이는 그의 아버지 후세인 이븐 무함마드 차으르 테긴이 언제인가 이 지역을 통치했기 때문이었다. 반면 고향인 카슈가리에 대해서는 그 주민들이 ‘최악의 인간’이라며 혹평을 쏟아내었는데, 이는 아마 그의 아버지가 카슈가리에서 정치적으로 어려움을 겪었던 것과 연관이 있을 것이다. \n\n알카슈가리의 학문 여정은 당대 지식인 호자 사이푸딘의 딸이었던 어머니, 뷔비 라비야의 교육에서 시작되었다. 1056년부터 1057년까지 카라한 왕조 내부에서 발생한 분란에 휩싸인 그는 고향을 떠나게 되었으나, 부하라와 이란, 이라크 등지를 돌아다니면서도 공부에 여념이 없었다. 그는 아랍어나 페르시아어에도 유창했다. 한편 저작들에서 보이는 다양한 튀르크어 사용 유목민들에 대한 서술로 볼때, 망명기간에 다양한 튀르크어 사용 유목민들이 거주하는 지역 각지를 여행하기도 했던 것으로 보인다\n\n이후 10년 간의 망명 생활의 끝에 바그다드로 이주하여 1072년부터 약 5년 간 튀르크어·아랍어 사전의 집필 작업에 돌입했다. 그 결과가 《튀르크어 대사전》으로, 마흐무드 알카슈가리는 이 책을 압바스 왕조의 칼리프 알무크타디에게 헌정했다\n\n신장 지역에서 발견된 문서들에 따르면 마흐무드 알카슈가리는 만년에 고향으로 돌아갔다. 이슬람력 477년, 즉 서기 1085년 ~ 1086년 사이에 죽었다고 전해진다. 알카슈가리의 무덤은 1981년 카슈가르에서 서쪽으로 45km 떨어진 오팔이라는 곳에서 발견되었다.",마흐무드 알카슈가리가 <튀르크어 대사전>을 만들기 시작한 년도는?,666,1072,52603,"Korean, English, Arabic"


### 6.년도, 해 로 끝나는 질문에 대한 답변
- 년도나 해를 묻는 답변에서는 년으로 끝나는게 맞아 보이지만 몇개는 `년`을 붙이지 않은 것이 몇개 인지 확인

In [123]:
years_df = train_df[train_df["question"].str.contains(r'년도|연도|해$', regex=True)]
print(len(years_df))
years_df.head()

92


,title,context,question,answer_start,answer_text,document_id,languages
13,장면,"장면은 귀국전 국무총리에 임명되었으나 이듬해인 1951년 2월에 귀국한 후 정식으로 취임하였다. 귀국 무렵 대한민국 국회와 이승만은 갈등하고 있었다. 그는 중간에서 양자의 조정 역할을 하려 했으나 실패하였다 그는 귀국 즉시 총리직에 취임하지 않고, 1주일 간의 여유를 얻어 요인들을 만나 의논해 보았다 그는 “일반적으로 이 박사에 대한 평이 좋지 않아 총리직을 맡을 생각이 간절하지 않았으나 이왕 인준도 받았으니 하는 데까지 하다가 할 수 없으면 그만두라”라는 주변의 의견대로 총리직을 맡게 되었다 당초 이승만은 장면에게 인사권을 위임하겠다고 하였으나, “5석 중 3석을 총리가 정하였으니 내무에는 이순용(李淳鎔), 국방에는 이기붕을 써 주시오”하고 종용하자, 당초 약속과는 달랐으나 장면은 받아들였다 국무총리 재임 중 그는 호화로운 식단을 기피하고 직접 도시락을 싸들고 출퇴근하였다.\n\n \n8월에는 자유당이 창당되자 자유당에 입당했다. 12월에는 원외 자유당이 결성되면서 장면이 속한 자유당은 원내 자유당으로 불리게 되었다. 원내 자유당에서는 1952년 제2대 정부통령 선거에서 그를 대통령 후보로 추대하려는 일파와, 내각 책임제로 개헌하여 대통령직을 유명무실화하게 한 뒤 그를 내각 책임제 하의 국무총리로 추대하려는 세력이 나타났다. 흥사단도 장면을 지지하였다. 이후 흥사단의 지지를 이유로 장면은 흥사단계로 분류되었으나 장면은 이를 부정하였다. 장면을 대통령 후보로 추대하려는 세력과 내각책임제 개헌 후 장면을 총리로 추대하려는 세력의 존재가 이승만의 귀에 들어가면서 장면은 정치적으로 궁지에 몰리게 된다.",장면이 한국으로 돌아온 년도는?,26,1951년,6930,"Korean, English, Chinese"
16,제1차 세계 대전,"양측 모두 2년 동안 서로에게 결정적인 타격을 줄 수 있는 공격을 하지 못했다. 1915~1917년 동안, 대영제국 및 프랑스는 전략, 전술적 방향의 측면의 선택 때문에 독일보다 더 많은 사상자로 고통받았다. 독일은 오직 하나의 주요 공세만 시도했지만 연합군은 독일의 방어선을 돌파하기 위한 여러 시도를 하였다.\n\n1916년 2월 독일은 프랑스의 베르됭에서 프랑스 진지를 공격하면서 베르됭 전투가 시작되었다. 1916년 12월 프랑스의 반격으로 공세 시작 지점 이전까지 독일군을 퇴각시키기 전까지 전투는 독일군이 많은 이익을 보았다. 프랑스군의 사상자는 엄청났지만, 독일군의 사상자 또한 이에 못지 않게 높아서 양 측 교전국의 사상자는 70만명 에서 97만 5천명 가까이 된다. 베르됭은 프랑스의 의지 및 희생의 상징이 되었다. \n\n1916년 7월부터 11월까지 영국-프랑스군은 솜 전투를 통해 독일군을 공격했다. 이 공격이 시작한 7월 1일 당시 영국 육군은 역사상 가장 큰 사상자가 발생해, 첫날 전투에만 19,240명이 사망하는 등 총 사상자가 75,470명이나 되었다. 솜 공세 전체 기간 동안 영국군의 사상자는 약 42만명이었다. 프랑스군 또한 대략 20만명의 사상자가 나왔으며, 독일군은 50만명의 사상자가 나왔다. \n\n1916년 전체 기간 동안 베르됭에서의 오래된 공격은 솜 전투의 심각한 사상자와 합쳐져 프랑스군이 지쳐 붕괴 위기까지 달했다. 1917년 4월 5일 니벨 공세가 연합국의 실패로 끝난 이후 영국 및 프랑스군의 헛된 정면 공격 시도는 프랑스 병사(poilu)의 프랑스군 반란으로 이어졌다. 동시에 일어난 영국의 아라스 전투는 제한된 범위에서 이루어져 궁극적으로 전략적 가치는 떨어졌지만 더 많은 성공을 거두었다. 아라스 공세의 일부분으로 캐나다 군단이 비미 리지를 점령했고, 이는 해당 국가에게 매우 중요하게 되었다. 캐나다의 국가정체성이 전투로 탄생되었다는 생각은 이후 캐나다의 군사 및 일반 사학계에서 널리 퍼지게 되는 의견이 되었다. \n\n1917년 7월부터 11월까지 이루어진 파스샹달 전투는 이 기간 벌어진 영국의 마지막 대규모 공세(프랑스의 지원)가 되었다. 이 공세는 10월 전장이 진흙탕으로 되기 전까지 연합국 사이의 약속으로 진행되었다. 사상자에 대해서는 많은 의견이 존재하지만 거의 20만명에서 40만명 사이로 추측한다.\n\n서부 전선의 참호전은 몇 년 동안 많은 지역의 점령이 이루어지지 않게 되었고, 그 결과 이 전선은 종종 정적이고 변하지 않는 전선이라고 생각한다. 그러나, 이 기간 동안 영국, 프랑스, 독일은 끊임없는 전술의 발전으로 인해 새로운 전장으로 나타났다.",영국군 약 42만명의 사상자가 발생한 전투가 시작된 연도는?,414,1916년,5510,"Korean, English"
22,호르스트 슈만,"그러다가 1944년에 아우슈비츠를 떠나 작센의 군 병원으로 이동한 슈만은 서부전선에서 군의관으로 일하다가 1945년 1월에 멩겔레와 비슷하게 미군의 포로가 되었다. 그러나 독일의 패전 후인 1945년 10월에 호르스트 슈만은 석방되었고,1946년에 그라트벡의 스포츠 관련 의사로 일하면서 전쟁이 끝난 후 한동안 별다른 일 없이 평안하게 지냈다, 그러나 1951년에 사냥 엽총면허증을 발급받으려고 제출한 신청서의 심사 과정에서 경찰 당국은 그가 수배 인물임을 파악하였고 체포영장이 발부되자 낌새를 느낀 그는 여권도 없이 곧바로 해외로 달아났다.\n\n호르스트 슈만은 처음에는 미국을 전전했다 호르스트 슈만이 한동안 어디에 있는지 파악이 되지 않았으나 우연한 기회에 밝혀지게 되었다. 독일에서 발간되는 ""그리스도와 세계""라는 잡지에 아프리카 가나의 오지에서 의학기술을 베푸는 '제2의 슈바이처 박사'라는 기사가 사진과 함께 소개된 것이 결정적이였다. 호르스트 슈만은 가나의 국가수반인 콰메 은크루마가 이끄는 은크루마 정권이 자신을 끝까지 지켜줄 것이라고 믿었지만 1966년 은크루마 정권이 무너지게 되자 곧바로 독일로 이송되었다. 결국 마지막엔 아프리카의 가나까지 달아나기도 했지만 결국 1966년에 가나에서 붙잡혀서 서독정부에 인계되었던 것이다.\n\n1970년 9월에 슈만의 재판이 시작되었다. 슈만은 잡히기 이전부터 이미 나치 독일의 강제수용소에서 1만 5천여명을 학살한 혐의가 있었던 강제수용소 의사로 재판을 받았는데, 호르스트 슈만은 24일 죽음의 감방에서 치명적인 일산화탄소 가스를 집어넣는 버튼을 자기가 눌렀었다고 자백했으며 그는 1만 4천 5백명의 장애인과 정신질환자인 독일 사람과 약 8백여명의 전쟁포로를 학살한 사실을 인정했다, 그러나 6년 간의 수감 생활 끝에 법원은 1972년 7월 29일에 건강상의 이유를 들어 호르스트 슈만을 형무소에서 풀려나게 했다, 의사들의 진단서에는 그의 병세가 매우 위중하다고 기록되어 있지만 그는 11년이 넘는 시간 동안 살았다고 한다. 그러다가 11년 후인 1983년 5월 5일 호르스트 슈만은 77세라는 나이로 천수를 누리고 사망했다.",슈만이 미군에게 포로로 잡혔던 년도는?,59,1945년,51125,"Korean, English"
32,외국의 시리아 내전 관여,"외국의 시리아 내전 관여는 2011년 3월 시작된 시리아 내전에 참여한 세력에 정치, 군사, 작전적인 지원을 하는 것과 외국의 활발한 개입을 의미한다. 시리아 내전에 참여한 대부분의 세력은 외국으로부터 다양한 종류의 지원을 받았다. 시리아 내전은 미국과 러시아, 그리고 이란과 사우디아라비아를 중심으로 한 역내 세력과 세계 강대국의 대리 전쟁이 중복되는 것으로 묘사되고 있다.. \n\n시리아 바트당 정부는 이란과 러시아에 군사 및 정치적인 지원을 받고 있으며, 레바논의 헤즈볼라와 시리아에 본부를 둔 팔레스타인 무장단체 PFLP-GC도 시리아 정부를 돕고 있다. 공개적이고 합법적으로 시리아에 기지를 보유한 유일한 해외 국가인 러시아는 시리아 정부의 요청에 따라 2015년 9월 30일부터 ISIL과 다른 반정부 단체에 맞서 공중전을 시작했다. 러시아의 군사 행위는 미국과 터키의 비난을 받았다. 터키는 2015년 11월부터 러시아 전투기가 영공을 침범했다는 이유로 요격하기도 했으며, 터키가 지원하는 반정부군에 러시아의 공격이 발생하자 이를 비판하기도 

In [126]:
# 년도 or 연도에 끝에 오는 질문의 답변 둘러보기
for years in years_df["answer_text"]:
  print(years)

1951년
1916년
1945년
2015년
1088년
1931년
1966년
1665년
1856년
1994년
1610년
1966년
2006년
1897년
1636년
2008년
1962년
1807년
1936년
1309년
1603년
1053
293년
1990년
1945년
2014
2004년
2008년
2008년
2015년
1992년
1014년
2004년
1826년
1951년
1801
1944년
1990년
1939년
1918년
2011년
1619년
1931년
1059년
1999년
1433년(세종 15년)
551년
2008년
1948년
1938년
1927년
2007년
2006년
2004년
1978년
2005년
1207년
1681년
미카와 국 다이주 사
1867년
2008년
2014년
1934년
824년
2006년
2005년
1946년
1560년
기원전 331년
1997년
893년
1928년
1920년
1755년
1515년
독일의 미드필더 토니 크로스
1072
1119년
2016년
1879년
1860년
1945년
1896년
1974년
2008년
1999년
1793
1945년
1502년
1362년
1949년
1978년


In [124]:
len(years_df[years_df["answer_text"].str.contains("년")])

85

# train 토큰화 데이터 분석
- `train` 파일에 대한 탐색 내용입니다
- 살펴본 내용은 다음과 같습니다.
1. 텍스트 토큰화
    - 대응되는 토큰들 확인
    - UNK 토큰 개수 확인 (Question, Context)
    - Unknown token외에 문제점이 보이는 token 확인
2. 일본어, 한자 토큰 확인
3. 특수문자 토큰 확인
4. 개행문자 토큰 확인



In [18]:
import transformers
from transformers import AutoTokenizer
from datasets import DatasetDict, load_from_disk, load_metric
from arguments import DataTrainingArguments, ModelArguments
import numpy as np
datasets = load_from_disk("../data/train_dataset")
print(datasets)

tokenizer = AutoTokenizer.from_pretrained("uomnf97/klue-roberta-finetuned-korquad-v2", use_fast=True)

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})


### 1.텍스트 토큰화

In [6]:
question = datasets["train"]["question"]
context = datasets["train"]["context"]

In [74]:
max_seq_length = 2500 # 겹치는 부분이 없도록, 데이터 확인을 위해서 크게 잡음
doc_stride = 128
tokenized_examples = tokenizer(
            question,
            context,
            truncation="only_second",
            max_length=max_seq_length,
            stride=doc_stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            return_token_type_ids=False, # roberta모델을 사용할 경우 False, bert를 사용할 경우 True로 표기해야합니다.
            padding="max_length"
        )

In [67]:
cls_token_id = tokenizer.cls_token_id
sep_token_id = tokenizer.sep_token_id
pad_token_id = tokenizer.pad_token_id
unk_token_id = tokenizer.unk_token_id
mask_token_id = tokenizer.mask_token_id

print(f"[CLS] token ID: {cls_token_id}")
print(f"[SEP] token ID: {sep_token_id}")
print(f"[PAD] token ID: {pad_token_id}")
print(f"[UNK] token ID: {unk_token_id}")
print(f"[MASK] token ID: {mask_token_id}")

[CLS] token ID: 0
[SEP] token ID: 2
[PAD] token ID: 1
[UNK] token ID: 3
[MASK] token ID: 4


In [69]:
# Decoding
example = tokenized_examples["input_ids"][3]
decoded_text = tokenizer.decode(example, skip_special_tokens=True)
decoded_text

'11 ~ 12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요? 불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든 것을 불감 ( 佛 ) 이라고 한다. 불감은 그 안에 모신 불상의 양식뿐만 아니라, 당시의 건축 양식을 함께 살필 수 있는 중요한 자료가 된다. n n이 작품은 높이 18cm의 작은 불감으로, 청동으로 불감과 불상을 만들고 그 위에 금칠을 하였다. 불감 내부를 살펴보면 난간을 두른 사각형의 기단 위에 본존불과 양 옆에 보살상이 있으며, 그 위에 기둥과 지붕으로 된 뚜껑이 덮혀 있다. 법당 모양의 뚜껑에는 앞면과 양쪽에 커다란 창문이 있어서 안에 모셔진 불상을 잘 볼 수 있도록 하였다. n n본존불은 얼굴이 추상적이고, 양 어깨를 감싸고 있는 옷은 주름을 간략한 선으로 표현했다. 몸 뒤편에 있는 광배 ( 光 ) 는 머리광배와 몸광배로 나누어져 있으며, 불꽃무늬로 가장자리를 장식하고 있다. 본존불 양 옆의 보살상도 구슬로 장식된 관 ( ) 을 쓰고 있다는 점을 제외하면 형식이나 표현 수법이 본존불과 유사하다. n n불감은 지금도 금색이 찬란하고 지붕에 녹청색이 남아 있는 등 전체적인 보존 상태가 양호하다. 본존불의 긴 허리, 불규칙하게 나타나는 옷주름, 그리고 보살이 쓰고 있는 구슬로 장식한 관 ( ) 등 여러 양식으로 보아 만든 시기는 중국 북방 계통의 영향을 받은 11∼12세기 경으로 추정된다. 이 작품은 고려시대 또는 그 이전의 목조건축 양식과 조각수법을 보여주는 귀중한 예라는 점에서 가치가 크다고 할 수 있다.'

In [70]:
np.array(tokenized_examples["input_ids"]).shape

(3952, 2500)

#### 대응되는 토큰들 확인

Special Tokens
* [CLS] token ID: 0
* [SEP] token ID: 2
* [PAD] token ID: 1
* [UNK] token ID: 3
* [MASK] token ID: 4

In [71]:
def convert_input_ids_to_tokens(tokenizer, tokenized_examples):
    tokenized_data = []
    for input_id_list in tokenized_examples["input_ids"]:
        # input_ids 리스트를 다시 토큰으로 변환
        tokens = tokenizer.convert_ids_to_tokens(input_id_list)
        tokenized_data.append(tokens)
    
    return tokenized_data

In [72]:
tokenized_data = np.array(convert_input_ids_to_tokens(tokenizer, tokenized_examples=tokenized_examples))

In [73]:
print(tokenized_examples["input_ids"][1])

[0, 3845, 2125, 4004, 2446, 2084, 7604, 2079, 25087, 2052, 860, 1644, 2073, 35, 2, 11, 5496, 2125, 11984, 11, 4013, 11, 6725, 2125, 11984, 11, 3604, 3845, 2125, 11984, 6233, 4561, 2496, 2259, 4342, 2259, 9746, 2440, 2104, 28674, 18, 22, 2232, 3665, 2104, 2165, 2069, 4372, 2088, 16, 26, 18, 3912, 23249, 2079, 4342, 2200, 4227, 2073, 7087, 6968, 2170, 4154, 19521, 16, 4227, 12617, 2079, 8400, 2116, 4751, 7488, 6815, 16, 4501, 16, 9583, 7606, 27135, 10035, 2226, 2116, 2116, 4605, 2496, 2259, 4342, 2507, 2088, 16, 3666, 2073, 4175, 3719, 3674, 2125, 3908, 2170, 3646, 2052, 6670, 2069, 3645, 1889, 2414, 4342, 2507, 2062, 18, 3727, 19011, 2440, 11358, 4077, 2506, 2079, 9698, 170, 3939, 2079, 3966, 171, 793, 3845, 2125, 3939, 2079, 3872, 2069, 4277, 7488, 2112, 16, 4078, 5496, 2125, 4004, 2446, 2084, 7604, 2138, 11750, 3845, 2125, 4004, 2446, 2084, 7604, 2079, 18773, 2052, 3622, 18, 4077, 2506, 2259, 7097, 2079, 4008, 2069, 3986, 2205, 2307, 3939, 2052, 30142, 2104, 3841, 2079, 4145, 2052, 21

#### UNK 토큰 개수 확인 (Question, Context) - Train dataset 

In [84]:
def check_unknown(tokenized_samples):
    # tokenized_samples = ["input_ids"] 리스트들
    unk_count_per_index = []
    for i,sample in enumerate(tokenized_samples):
        sample_unk_count = 0
        for token in sample:
            if token == 3: # unk token ids
                sample_unk_count += 1
        
        unk_count_per_index.append([i, sample_unk_count])

    return np.array(unk_count_per_index)

In [85]:
unk_token = check_unknown(tokenized_examples["input_ids"]) # [질문,지문]별 UNK 개수

In [95]:
# 전체 context에 대한 UNK Token
print(np.sum(unk_token[:,1],axis=0))

28343


#### Unknown token외에 문제점이 보이는 token 확인

In [133]:
special_tokens = ["[CLS]","[PAD]","[SEP]","[MASK]"]
def get_encoded_data(data, index):
    tokens = []
    for i,token in enumerate(data[index]):
        if token in special_tokens:
            continue
        tokens.append(token)
    return tokens

In [172]:
special_tokens = ["[CLS]","[PAD]","[SEP]","[MASK]"] 
print(datasets["train"]["question"][0], datasets["train"]["context"][0])
print(",".join(get_encoded_data(tokenized_data, 0)))
print("number of UNK token :", unk_token[0]) # 총 4개

대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은? 미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다.\n\n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다.\n\n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05
대통령,##을,포함,##한,미국,##의,행정부,견제,##권,##을,갖,##는,국가,기관,##은,?,미국,상의,##원,또는,미국,상원,(,United,State,##s,Se,##n,##at

### 2.일본어, 한자 확인

In [173]:
from pprint import pprint
# 예시 데이터
pprint(datasets["train"]["question"][4].split("."))
pprint(datasets["train"]["context"][4].split("."))
print("----Tokenize----")
pprint(",".join(get_encoded_data(tokenized_data, 4)))

['명문이 적힌 유물을 구성하는 그릇의 총 개수는?']
['동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기의 유물로, 경상남도 울주군 웅촌면 대복리에서 '
 '출토되었다고 전한다',
 ' 외함(外函)은 청화명문이 있는 백자이며, 그 안쪽에 납작한 금속제 원형 합 2점과 금속제 원통형 합 등 3점의 그릇이 봉안되어 있다',
 '\\n\\n바깥쪽의 외함인 백자 합 동체 중앙부 표면에 청화안료로 쓴 “癸巳二月日 施主承表 兩主”라는 명문이 세로로 세 줄에 걸쳐서 쓰여 '
 '있어 조선 후기인 계사년에 시주자인 승표 부부가 발원하여 만든 것임을 알 수 있다',
 '\\n\\n동아대학교박물관의 계사명 사리구는 정확한 제작연대는 알 수 없지만 명문 등을 통해 적어도 17세기 이후에 제작된 것으로 '
 '추정되는 작품으로, 명문이 있는 조선 후기 경상도 지역 출토 사리장엄구라는 점에서 중요한 가치를 지닌 작품으로 판단된다',
 '\\n\\n조선 후기 사리장엄구는 아직까지 조사와 연구가 거의 이루어지지 않았으나, 이처럼 세트를 갖추어 출토된 유물은 비교적 드문 '
 '편임을 고려할 때, 이 계사명 사리장엄구는 제작연대와 발원자의 이름이 밝혀져 있으며, 지금까지 출토된 예가 드문 비교적 완전한 세트를 '
 '가진 유물이라는 점에서 조선 후기 사리장엄구 연구에 자료적 가치를 지닌 유물이다',
 '']
----Tokenize----
'명문,##이,적힌,유물,##을,구성,##하,##는,그릇,##의,총,개수,##는,?,동아,##대,##학교,##박,##물,##관,##에서,소장,##하고,있,##는,계,##사,##명,사리,##구,##는,총,4,##개,##의,용기,##로,구성,##된,조선,##후,##기,##의,유물,##로,,,경상남도,울주군,웅,##촌,##면,대,##복,##리에,##서,출토,##되,##었,##다고,전한다,.,외,##함,(,外,[UNK],),은,청,##화,##명,##문,##이,있,##는,백자,##이,##며,,,그,안쪽,##에,납작,##한,금속

In [174]:
# 해당 예시에 대한 unk 개수
unk_token[4]

array([ 4, 12])

한자를 대부분 잘 못알아 보는 것을 확인할 수 있다. 그렇다면 이 한자를 어떻게 해야하는가..? Answer에 한자가 포함된 경우를 보자

In [175]:
datasets["train"]["answers"][136]

{'answer_start': [202], 'text': ['‘전의식’(前意識)']}

### 3.특수문자 확인

In [176]:
from pprint import pprint
# 예시 데이터
pprint(datasets["train"]["question"][136].split("."))
pprint(datasets["train"]["context"][136].split("."))
print("----Tokenize----")
pprint(",".join(get_encoded_data(tokenized_data, 136)))

['프로이트의 정신 체계에서 현실적인 정신체계와 본능적 충동의 체계를 제외한 것의 명칭은?']
['프로이트는 꿈을 해석해서 참다운 꿈 생각을 밝혀낼 수 있다고 믿었다',
 ' 또한 환자와 허물없는 대화를 나누는 자유연상법에 따라 환자의 말을 해석함으로써 환자를 정신적 억압으로부터 해방시키고 치료할 수 있다고 '
 '믿었다',
 ' 반복 강박을 비롯한 삶의 충동과 죽음의 충동에 대한 예리한 통찰을 바로 이 책을 통해 엿볼 수 있다',
 '\\n\\n우선 프로이트는 정신의 체계를 ‘의식’, ‘전의식’(前意識), ‘무의식’의 세 가지로 구분하여 보았다',
 ' 의식은 원래 의식된 것으로서 이성적, 합리적, 현실적인 정신의 체계에 해당한다',
 ' 무의식은 의식되지 않은 것으로서 정신 과정의 대부분을 차지하는 무의식적인 본능적 충동의 체계다',
 ' 그런가 하면 전의식은 의식되기 이전의 정신 체계로서 무의식을 걸러서 의식 쪽으로 보내는 역할, 곧 검열을 행하는 정신의 체계다',
 '\\n\\n≪꿈의 해석≫에서 프로이트는 ‘의식’, ‘전의식’, ‘무의식’에 관해서 아직 철저하게 해명하지 못하고 있다',
 ' 따라서 그는 이 책 아울러 1923년에 출판한 ≪자아와 이드≫에서 정신 과정을 보다 더 명확하고 철저하게 밝힌다',
 ' 이 책에서 프로이트는 ‘의식’, ‘전의식’, ‘무의식’을 하나의 의식이라고 말한다',
 ' 곧 의식의 가장 많은 부분을 무의식이 차지하고 있고 가장 적은 부분을 전의식이 차지하고 있으며 이성적 현실 의식 역시 부분적이라는 '
 '것이다',
 ' 그런가 하면 ≪자아와 이드≫에서 프로이트는 정신 과정을 ‘원초아’, ‘자아’, ‘초자아’로 구분하는데 이러한 구분은 이 책에서의 정신 '
 '과정을 한층 더 역동적으로 밝히고 있다',
 ' 본능 충동으로서의 원초아와 도덕 및 양심에 관계되는 초자아는 무의식에 해당하고 현실적 이성 활동은 자아에 속한다',
 ' ≪자아와 이드≫에서 프로이트는 에로스와 타나토스, 곧 ‘사랑의 충동’과 ‘죽음의 충동’을 대립시키

위 데이터를 보면 확실히 한자에 대한 tokenizing이 제대로 이루어지지 않음을 볼 수 있다.

이렇다면 생각나는 문제점은.. 학습이 제대로 되지않아 만약에 한자가 또 들어오면 정답을 제대로 못낼 수 있다는 단점이 있다..

오히려 특수문자, "<< >>"는 제대로 Tokenizing하는 것을 볼 수 있음

만약에 답이 특수문자가 되는 경우에는 어떻게 나오는가?

In [178]:
from pprint import pprint
# 예시 데이터
pprint(datasets["train"]["question"][183].split("."))
pprint(datasets["train"]["context"][183].split("."))
print("----Tokenize----")
pprint(", ".join(get_encoded_data(tokenized_data, 183)))

['천연두에 감염된 모든 사례 중 출혈성 천연두는 얼마의 비중을 차지하는가?']
['출혈성 천연두는 피부, 점막, 소화관에 광범한 내출혈을 동반하는 심각한 천연두 유형이다',
 ' 출혈성 천연두는 전체 천연두 감염 사례의 약 2% 정도였으며, 대부분 성인에게 발병했다 출혈성 천연두의 발진은 수포를 형성하지 않고 '
 '부드러운 상태로 남아 있다',
 ' 대신 피부 아래로 출혈이 일어나서 마치 새까맣게 탄 것처럼 검은색을 띤다 그래서 이 유형의 천연두를 검은마마(black pox)라고도 '
 '했다\\n\\n감염 초기, 또는 전격형(fulminating) 출혈성 천연두는, 감염 이틀째 내지 사흘째가 되면 결막하 출혈로 인해 눈의 '
 '흰자위가 심홍색이 되면서 출혈이 확인된다',
 ' 출혈성 천연두는 어스름한 홍반, 점상출혈 및 비장, 신장, 장막, 근육에 출혈을 일으키고, 드물지만 외심막, 간, 고환, 난소, '
 '방광에도 출혈을 일으킨다',
 ' 환자는 보통 감염 5 ~ 7일째 경미해 보이는 피부 발진만을 남기고 갑자기 죽는다',
 ' 전격형 출혈성 천연두는 8 ~ 10일 정도 생존한 환자들에게서 나타났다',
 ' 발진 초기에 출혈이 나타나며, 발진은 도드라지지 않고 평평하고 수포 단계 이상 발달하지 않는다 출혈성 천연두 초기 환자는 혈중 '
 '응고인자(e',
 'g',
 ' 혈소판, 프로트롬빈, 글로불린)가 감소하고 안티트롬빈이 증가한다',
 ' 말기 환자는 확연한 혈소판감소증을 나타내지만 응고인자 감소는 덜 심하다',
 ' 말기 단계에서는 안티트롬빈 증가도 나타났다 천연두 유형의 병독성에 따라 이 유형의 천연두는 치명적 천연두 감염 사례의 3 ~ 25% '
 '정도로 잡힌다 출혈성 천연두는 대개 사망에 이를 정도로 치명적이다']
----Tokenize----
('천연, ##두, ##에, 감염, ##된, 모든, 사례, 중, 출혈, ##성, 천연, ##두, ##는, 얼마, ##의, 비중, ##을, '
 '차지, ##하, ##는, ##가, ?, 출혈, ##

%나 ()과 같은 특수문자에 대해서는 크게 문제가 되지 않을 것 같음

### 4.\n 개행문자 토큰 확인

위 예시를 보았을 때 "\"에 해당하는 부분이 [UNK]로 인식하여, "\\\n"을 개행토큰으로 보지 못한다.  

In [189]:
def get_newline(data_samples):
    all_lines = []
    for i,data in enumerate(data_samples):
        new_line_count = data.count("\\n")
        all_lines.append([i, new_line_count])
    return all_lines


In [193]:
newline_counts = np.array(get_newline(datasets["train"]["context"]))

In [200]:
newline_counts

array([[   0,    4],
       [   1,    2],
       [   2,    5],
       ...,
       [3949,   16],
       [3950,   14],
       [3951,    6]])

# 데이터셋 탐색적 분석
- `train` 파일에 대한 탐색 내용입니다
- 살펴본 내용은 다음과 같습니다.
1. 데이터 로드
2. 텍스트 정규화 및 분석
3. 성능 평가 메트릭
4. 데이터 분석 및 출력
5. 메인 실행
6. 통합
7. 성능 평가
8. 파인튜닝 결과

In [53]:
import json
from sklearn.metrics import f1_score
from collections import *
import re

  ### 1.데이터 로드

In [54]:
def load_project_dataset(filepath): #
  with open(filepath, 'r') as f:
    data = json.load(f)

  print(data.keys())

  return data

def load_korquad_dataset(filepath): #
  with open(filepath, 'r') as f:
    data = json.load(f)

  print(data.keys())

  if "data" in data:
    train_data = [item for topic in data['data'] for item in topic['paragraphs'] ]

    print(train_data)

  return data

### 2.텍스트 정규화 및 분석

In [55]:
def normalize_text(text):
    # 소문자 변환
    text = text.lower()
    # 특수 문자 제거
    text = re.sub(r"[^a-zA-Z0-9가-힣\s]", "", text)
    # 공백 정리
    text = re.sub(r"\s+", " ", text).strip()
    return text

def special_only_text(text):
  # 소문자 변환
  text = text.lower()
  # 알파벳 한글 숫자 제거
  text = re.sub(r"[a-zA-Z0-9가-힣]", "", text)
  # 공백 정리
  text = re.sub(r"\s+", " ", text).strip()
  return text

def normalize_context(data):
  context_special_only = []
  for context in data['context']:
    context = normalize_text(context).split()
    context_special_only += context

  print("normalized context:",Counter(context_special_only).most_common(10000))

def special_only_context(data):
  context_special_only = []
  for context in data['context']:
    context = special_only_text(context).split()
    context_special_only += context

  print("context 내 특수문자 빈도:",Counter(context_special_only).most_common(10000))

def normalize_answer(data):
  answer_normal = []
  for answer in data['answers']:
    answer = normalize_text(answer['text'][0]).split()
    answer_normal += answer

  print("normalized answer:", Counter(answer_normal).most_common(10000))

def special_only_answer(data):
  answer_special_only = []
  for answer in data['answers']:
    answer = special_only_text(answer['text'][0]).split()
    answer_special_only += answer

  print("answer 내 특수문자 빈도:", Counter(answer_special_only).most_common(10000))

### 3.성능 평가 메트릭

In [56]:
def compute_f1(pred, ground):
  pred_tokens = pred.split()
  ground_tokens = ground.split()
  common1 = Counter(pred_tokens) & Counter(ground_tokens)
  common2 = set(pred_tokens) & set(ground_tokens)
  num_common = sum(common1.values())

  if num_common == 0: return 0.0

  precision = num_common / len(pred_tokens)
  recall = num_common / len(ground_tokens)

  f1 = 2 * precision * recall / (precision + recall)
  return f1

def compute_em(pred, ground):
  return int(pred == ground)

def evaluate_mrc(data):
  em_scores = []
  f1_scores = []

  for answer, context in zip(data['answers'], data['context']):
    pred = context[answer['answer_start'][0]:answer['answer_start'][0] + len(answer['text'][0])]
    ground = answer['text'][0]

    em_scores += [compute_em(pred, ground)]
    f1_scores += [compute_f1(pred, ground)]

  print(f"average em score: {sum(em_scores) / len(em_scores) * 100:.2f}")
  print(f"average f1 score: {sum(f1_scores) / len(f1_scores) * 100:.2f}")

### 4.데이터 분석 및 출력

In [57]:
def print_data(data):
  print(f"dataset keys: {data.keys()}")
  #print(f"dataset version: {data.version}")
  print(f"dataset size: {len(data['title'])}")

  context_count = len(data['context'])#[len(article['paragraphs']) for article in data]
  #print(f"paragraphs per doc: {len(context_count)}")
  print(f"number of article: {context_count}")

  question_count = len(data['question'])#[len(article['paragraphs']) for article in data]
  #print(f"paragraphs per doc: {len(context_count)}")
  print(f"number of question: {question_count}")

def print_text_info(data, column_name):
  num_of_words = []
  seq_length = []

  if type(data[column_name][0]) is not str:
    print("The field data is not String")
    return

  for seq in data[column_name]:
    num_of_words += [len(seq.split())]
    seq_length += [len(seq)]

  print(f"average {column_name} num of words: {sum(num_of_words) / len(num_of_words):.2f}")

  print(f"max {column_name} num of words: {max(num_of_words):.2f}")

  print(f"average {column_name} length: {sum(seq_length) / len(seq_length):.2f}")

  print(f"max {column_name} length: {max(seq_length):.2f}")

  words = [word for seq in data[column_name] for word in seq.split()]
  print(f"{column_name} most common word: {Counter(words).most_common(30)}")

  first_word = [seq.split()[0] for seq in data[column_name]]
  print(f"{column_name} most common first word: {Counter(first_word).most_common(30)}")

  last_word = [seq.split()[-1] for seq in data[column_name]]
  print(f"{column_name} most common last word: {Counter(last_word).most_common(30)}")

def print_questions(data):
  print_text_info(data, 'question')

def print_contexts(data):
  print_text_info(data, 'context')

def print_answers(data):
  answer_num_of_words = []
  answer_word_position =[]

  answer_length = []
  answer_char_position =[]

  column_name = 'answers'

  for answer, context in zip(data['answers'], data['context']):
    answer_text = answer['text'] # " ["tessdfjksfj"]"
    answer_start = answer['answer_start']# "[1233]"
    if len(answer_text) > 1:print("multiple answer")
    if len(answer_start) > 1:print("multiple answer start")
    if answer_text[0] == "":print("@@@@@ no answer")
    if answer_text[0].lower() in ["null", "no", "", "none", None]:print("@@@@@@ no  answer")
    #if context[answer_start[0]] == answer_text[0]:print("yes");print(context[answer_start[0]], answer_start[0], answer_text, answer_text[0])
    #else:print('no');print(context[answer_start[0]], answer_start[0], answer_text, answer_text[0])

    answer = answer_text[0]
    answer_pos = answer_start[0]

    answer_num_of_words += [len(answer.split())]
    answer_length += [len(answer)]

    answer_word_position += [len(context[:answer_pos+1].split())]
    answer_char_position += [answer_pos]

  print(f"average {column_name} num of words: {sum(answer_num_of_words) / len(answer_num_of_words):.2f}")

  print(f"max {column_name} num of words: {max(answer_num_of_words):.2f}")

  print(f"average {column_name} length: {sum(answer_length) / len(answer_length):.2f}")

  print(f"max {column_name} length: {max(answer_length):.2f}")

  print(f"average {column_name} word start position: {sum(answer_word_position) / len(answer_word_position):.2f}")

  print(f"max {column_name} word start position: {max(answer_word_position):.2f}")

  print(f"average {column_name} char start position: {sum(answer_char_position) / len(answer_char_position):.2f}")

  print(f"max {column_name} char start position: {max(answer_char_position):.2f}")

  answer_word = [word for answer in data['answers'] for word in answer['text'][0].split()]
  print(f"answer most common word: {Counter(answer_word).most_common(30)}")

  answer_first_word = [word[0] for answer in data['answers'] for word in answer['text'][0].split()]
  print(f"answer most common first word: {Counter(answer_first_word).most_common(30)}")

  answer_last_word = [word[-1] for answer in data['answers'] for word in answer['text'][0].split()]
  print(f"answer most common last word: {Counter(answer_last_word).most_common(30)}")

### 5.메인 실행

In [58]:
if __name__ == "__main__":
  filepath = "train_examples.json"
  data = load_project_dataset(filepath)

  print("data analysis")
  print_data(data)

  print("question analysis")
  print_questions(data)

  print("context analysis")
  print_contexts(data)
  normalize_context(data)
  special_only_context(data)

  print("answer analysis")
  print_answers(data)
  normalize_answer(data)
  special_only_answer(data)

  print("metric analysis")
  evaluate_mrc(data)

dict_keys(['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'])
data analysis
dataset keys: dict_keys(['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'])
dataset size: 3952
number of article: 3952
number of question: 3952
question analysis
average question num of words: 6.96
max question num of words: 19.00
average question length: 29.32
max question length: 78.00
question most common word: [('사람은?', 224), ('인물은?', 207), ('것은?', 187), ('있는', 178), ('수', 174), ('어떤', 171), ('위해', 148), ('것은', 138), ('무엇인가?', 128), ('누구인가?', 125), ('된', 122), ('해는?', 118), ('곳은?', 115), ('어느', 103), ('중', 100), ('한', 96), ('몇', 85), ('가장', 84), ('무엇을', 80), ('이름은?', 73), ('때', 73), ('어디인가?', 72), ('함께', 64), ('당시', 64), ('만든', 62), ('누구의', 59), ('하는', 54), ('인물은', 53), ('언제', 50), ('사람은', 45)]
question most common first word: [('윤치호가', 17), ('교황', 9), ('미국', 8), ('윌리엄', 7), ('제2차', 6), ('요한', 6), ('현재', 6), ('기원전', 5), ('최초의', 5), ('일본이'

In [59]:
if __name__ == "__main__":
  filepath = "dev_examples.json"
  data = load_project_dataset(filepath)

  print("data analysis")
  print_data(data)

  print("question analysis")
  print_questions(data)

  print("context analysis")
  print_contexts(data)
  normalize_context(data)
  special_only_context(data)

  print("answer analysis")
  print_answers(data)
  normalize_answer(data)
  special_only_answer(data)

  print("metric analysis")
  evaluate_mrc(data)

dict_keys(['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'])
data analysis
dataset keys: dict_keys(['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'])
dataset size: 240
number of article: 240
number of question: 240
question analysis
average question num of words: 6.97
max question num of words: 15.00
average question length: 29.20
max question length: 59.00
question most common word: [('것은?', 18), ('것은', 15), ('있는', 13), ('어떤', 9), ('누구인가?', 9), ('사람은?', 9), ('인물은?', 9), ('위해', 8), ('수', 8), ('무엇인가?', 8), ('어디인가?', 7), ('곳은?', 7), ('대한', 6), ('중', 6), ('다른', 5), ('이름은?', 5), ('몇', 5), ('무엇을', 5), ('준', 5), ('하는', 5), ('사용한', 4), ('어느', 4), ('방법은?', 4), ('해는?', 4), ('시기는?', 4), ('무엇의', 4), ('함께', 4), ('의해', 4), ('누구의', 4), ('때', 4)]
question most common first word: [('김준연은', 2), ('처음으로', 1), ('스카버러', 1), ('촌락에서', 1), ('로타이르가', 1), ('의견을', 1), ('1945년', 1), ('징금수는', 1), ('다른', 1), ('루이', 1), ('헤자즈', 1), ('버드', 1), ("

### 6.통합

In [60]:
import json
import re
#import matplotlib.pyplot as plt
from collections import Counter
from transformers import BertTokenizer

# 1. 데이터 로드
def load_korquad_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

# 2. 데이터 전처리 - 텍스트 정규화
def normalize_text(text):
    text = text.lower()  # 소문자 변환
    text = re.sub(r"\s+", " ", text).strip()  # 불필요한 공백 제거
    return text

def print_data(data):
  print(f"dataset keys: {data.keys()}")
  #print(f"dataset version: {data.version}")
  print(f"dataset size: {len(data['title'])}")

  context_count = len(data['context'])#[len(article['paragraphs']) for article in data]
  #print(f"paragraphs per doc: {len(context_count)}")
  print(f"number of article: {context_count}")

  question_count = len(data['question'])#[len(article['paragraphs']) for article in data]
  #print(f"paragraphs per doc: {len(context_count)}")
  print(f"number of question: {question_count}")

def print_questions(data):
  context_length = []
  question_length = []

  for context in data['context']:
    context_length += [len(context.split())]

  for question in data['question']:
    question_length += [len(question.split())]

  print(f"average paragraph length: {sum(context_length) / len(context_length):.2f}")

  print(f"max paragraph length: {max(context_length):.2f}")

  print(f"average question length: {sum(question_length) / len(question_length):.2f}")

  print(f"max question length: {max(question_length):.2f}")

  question_word = [word for question in data['question'] for word in question.split()]
  print(f"question most common word: {Counter(question_word).most_common(30)}")

  question_first_word = [question.split()[0] for question in data['question']]
  print(f"question most common first word: {Counter(question_first_word).most_common(30)}")

  question_last_word = [question.split()[-1] for question in data['question']]
  print(f"question most common last word: {Counter(question_last_word).most_common(30)}")

def print_answers(data):
  answer_length = []
  answer_position =[]

  for answer, context in zip(data['answers'], data['context']):
    answer_text = answer['text']
    answer_start = answer['answer_start']
    if len(answer_text) > 1:print("multiple answer")
    if len(answer_start) > 1:print("multiple answer start")
    if answer_text[0] == "":print("@@@@@ no answer")
    if answer_text[0].lower() in ["null", "no", "", "none", None]:print("@@@@@@ no  answer")
    #if context[answer_start[0]] == answer_text[0]:print("yes");print(context[answer_start[0]], answer_start[0], answer_text, answer_text[0])
    #else:print('no');print(context[answer_start[0]], answer_start[0], answer_text, answer_text[0])
    answer_length += [len(answer_text[0].split())]
    answer_position += [answer_start[0]]

  print(f"average answer length: {sum(answer_length) / len(answer_length):.2f}")
  print(f"average answer position: {sum(answer_position) / len(answer_position):.2f}")

  answer_word = [word for answer in data['answers'] for word in answer['text'][0].split()]
  print(f"answer most common word: {Counter(answer_word).most_common(30)}")

  answer_first_word = [word[0] for answer in data['answers'] for word in answer['text'][0].split()]
  print(f"answer most common first word: {Counter(answer_first_word).most_common(30)}")

  answer_last_word = [word[-1] for answer in data['answers'] for word in answer['text'][0].split()]
  print(f"answer most common last word: {Counter(answer_last_word).most_common(30)}")

def normalize_special_text(text):
    # 소문자 변환
    text = text.lower()
    # 특수 문자 제거
    text = re.sub(r"[^a-zA-Z0-9가-힣\s]", "", text)
    # 공백 정리
    text = re.sub(r"\s+", " ", text).strip()
    return text

def special_only_text(text):
  # 소문자 변환
  text = text.lower()
  # 특수 문자 이외 제거
  text = re.sub(r"[a-zA-Z0-9가-힣]", "", text)
  # 공백 정리
  text = re.sub(r"\s+", " ", text).strip()
  return text

def special_only_context(data):
  context_special_only = []
  for context in data['context']:
    context = special_only_text(context).split()
    context_special_only += context

  print(Counter(context_special_only).most_common(10000))

def normal_answer(data):
  answer_normal = []
  for answer in data['answers']:
    answer = normalize_text(answer['text'][0]).split()
    answer_normal += answer

  print("normalized answer:", Counter(answer_normal).most_common(10000))

def special_only_answer(data):
  answer_special_only = []
  for answer in data['answers']:
    answer = special_only_text(answer['text'][0]).split()
    answer_special_only += answer

  print("answer 내 특수문자 빈도:", Counter(answer_special_only).most_common(10000))

# 3. 데이터 분석 준비
def preprocess_korquad1(data):
    dataset = []
    for article in data['data']:
        for paragraph in article['paragraphs']:
            context = normalize_text(paragraph['context'])
            for qa in paragraph['qas']:
                question = normalize_text(qa['question'])
                if len(qa['answers']) > 0:
                    answer_text = normalize_text(qa['answers'][0]['text'])
                    answer_start = qa['answers'][0]['answer_start']
                    dataset.append((context, question, answer_text, answer_start))
    return dataset

# 3. 데이터 분석 준비
def preprocess_korquad(data):
    contexts = []
    answers = []
    questions = []
    titles = []

    for article in data['data']:
        for paragraph in article['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                question = qa['question']
                if len(qa['answers']) > 0:
                    answer_text = qa['answers'][0]['text']
                    answer_start = qa['answers'][0]['answer_start']
                else:print("qas answers len < 0 what the?")

                contexts += [context]
                answers += [{'text':[answer_text], 'answer_start':[answer_start]}]
                questions += [question]
                titles += [article['title']]


    dataset = {'title':titles, 'context':contexts, 'question':questions, 'answers': answers}
    return dataset

# 4. 기본 통계 분석
def analyze_data(dataset):
    context_lengths = [len(context) for context, _, _, _ in dataset]
    question_lengths = [len(question) for _, question, _, _ in dataset]
    answer_lengths = [len(answer) for _, _, answer, _ in dataset]

    # 평균 길이 출력
    print(f"문맥의 평균 길이: {sum(context_lengths) / len(context_lengths):.2f}")
    print(f"질문의 평균 길이: {sum(question_lengths) / len(question_lengths):.2f}")
    print(f"정답의 평균 길이: {sum(answer_lengths) / len(answer_lengths):.2f}")

    # 최대, 최소 길이
    print(f"문맥의 최대 길이: {max(context_lengths)}")
    print(f"문맥의 최소 길이: {min(context_lengths)}")
    print(f"질문의 최대 길이: {max(question_lengths)}")
    print(f"질문의 최소 길이: {min(question_lengths)}")
    print(f"정답의 최대 길이: {max(answer_lengths)}")
    print(f"정답의 최소 길이: {min(answer_lengths)}")

    # 길이 분포를 그래프로 시각화
    plt.figure(figsize=(12, 6))
    plt.hist(context_lengths, bins=50, alpha=0.7, label='Context Length')
    plt.hist(question_lengths, bins=50, alpha=0.7, label='Question Length')
    plt.hist(answer_lengths, bins=50, alpha=0.7, label='Answer Length')
    plt.legend()
    plt.title('Length Distribution of Context, Question, and Answer')
    plt.xlabel('Length')
    plt.ylabel('Frequency')
    plt.show()

# 5. 토크나이저 적용 및 토큰화
def tokenize_data(dataset):
    tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
    tokenized_dataset = []

    for context, question, answer, _ in dataset:
        context_tokens = tokenizer.tokenize(context)
        question_tokens = tokenizer.tokenize(question)
        answer_tokens = tokenizer.tokenize(answer)
        tokenized_dataset.append((context_tokens, question_tokens, answer_tokens))

    return tokenized_dataset

# 6. 빈도 분석
def analyze_token_frequencies(tokenized_dataset):
    context_tokens = []
    question_tokens = []
    answer_tokens = []

    for context, question, answer in tokenized_dataset:
        context_tokens.extend(context)
        question_tokens.extend(question)
        answer_tokens.extend(answer)

    # 가장 자주 등장하는 토큰 분석
    context_counter = Counter(context_tokens)
    question_counter = Counter(question_tokens)
    answer_counter = Counter(answer_tokens)

    print(f"문맥에서 가장 자주 등장하는 토큰: {context_counter.most_common(10)}")
    print(f"질문에서 가장 자주 등장하는 토큰: {question_counter.most_common(10)}")
    print(f"정답에서 가장 자주 등장하는 토큰: {answer_counter.most_common(10)}")

    # 토큰 빈도 그래프 그리기
    plt.figure(figsize=(12, 6))
    context_top_tokens = context_counter.most_common(10)
    plt.bar([token[0] for token in context_top_tokens], [token[1] for token in context_top_tokens], label="Context Tokens")
    plt.title("Top 10 Tokens in Context")
    plt.show()

# KorQuAD 데이터 파일 경로 설정
file_path = 'KorQuAD_v1.0_train.json'  # 적절한 파일 경로로 변경 필요

# 실행
korquad_data = load_korquad_data(file_path)
preprocessed_data = preprocess_korquad(korquad_data)
data = preprocessed_data
if preprocessed_data:
  print("data analysis")
  print_data(data)

  special_only_context(data)

  print("question analysis")
  print_questions(data)

  print("answer analysis")
  print_answers(data)

  special_only_answer(data)

  print("metric analysis")
  evaluate_mrc(data)
#analyze_data(preprocessed_data)

#tokenized_data = tokenize_data(preprocessed_data)
#analyze_token_frequencies(tokenized_data)


data analysis
dataset keys: dict_keys(['title', 'context', 'question', 'answers'])
dataset size: 60407
number of article: 60407
number of question: 60407
[('.', 421899), (',', 292188), ('"', 50247), (')', 32576), ('(', 31733), ("'", 23326), ('()', 15845), ('《', 14664), ('》', 11851), ('-', 10548), ('·', 8751), ('《》', 8737), ('“', 7961), ('〈', 6294), ("''", 6233), ('”', 5878), ('‘', 5287), ('〉', 5101), ('~', 5043), ('),', 4586), (':', 4448), ('’', 4376), ('""', 3950), ('."', 3857), ('%', 3298), ('(),', 2584), ('‘’', 2503), ('(,', 2326), ('/', 2289), ('(:', 2082), ('〈〉', 2077), (').', 1977), ('",', 1631), ('.%', 1593), ('.”', 1534), ('.)', 1436), ('<', 1376), ('..', 1306), ('··', 1248), ('&', 1098), ('>', 1058), ('.,', 1016), (',,', 920), ('《》,', 900), ('》(', 811), ('》,', 790), ("',", 788), ('.(', 764), ('〉,', 731), ('(.', 707), ('(-)', 679), ('"(', 660), ('...', 655), ('<>', 637), ('+', 579), ('》()', 559), ('-,', 544), (".'", 529), ('.%,', 509), ('!', 507), ('《》()', 501), ('「', 487), (';

### 7.성능 평가

In [61]:
filepath = "train_examples.json"
data = load_project_dataset(filepath)

filepath = "dev_examples.json"
data = load_project_dataset(filepath)

preds = ""
file_path = 'predictions_train.json'
with open(file_path, 'r', encoding='utf-8') as f:
  preds = json.load(f)

print(preds)
print("compare")
dt=[]
for id, pred in preds.items():
  i = data['id'].index(id)
  ground = data['answers'][i]['text'][0]
  f1 = compute_f1(pred, ground)
  em = compute_em(pred, ground)
  print(f"f1: {f1:.2f}, em: {em:.2f}")
  if em < 1:
    dt+=[[id,i,data['question'][i],pred,ground]]

print("----")
for d in dt:
  print(f"qusetion: {d[2]}, ground: {d[4]}, pred: {d[3]}")




dict_keys(['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'])
dict_keys(['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'])
{'mrc-0-003264': '서울지방법원 민사합의17부', 'mrc-0-004762': '1871년', 'mrc-1-001810': '나뭇잎', 'mrc-1-000219': '금대야', 'mrc-1-000285': '수평적 관계', 'mrc-0-005106': '쇼와 천황의 옥음방송', 'mrc-0-002076': '코칭 스티치', 'mrc-1-000414': '복잡한 감염병', 'mrc-0-002875': '스페인 펠리페 4세의 딸이었던 마리아 테레사가 프랑스', 'mrc-0-003828': '20세기 초', 'mrc-0-002778': '"5월의 왕"', 'mrc-0-003931': "'일급 비밀 프로젝트 2501'", 'mrc-0-002485': '테헤란', 'mrc-0-004483': '역사교육과정개발추진위원회', 'mrc-0-003032': '1967년 8월 16일', 'mrc-1-000724': '1963년', 'mrc-0-002138': '쿠빌라이', 'mrc-0-003727': '〈중앙일보〉', 'mrc-0-003115': '이이노야 성', 'mrc-0-003088': '전체 4순위', 'mrc-0-005296': '뇌물', 'mrc-1-001495': '준융합성 천연두', 'mrc-0-005289': '데스탱 장군', 'mrc-0-002240': '박트리아', 'mrc-0-000248': '기와조각', 'mrc-0-001846': '《국가》', 'mrc-1-001653': '3월 19일', 'mrc-0-000785': '브라질 포르투갈어', 'mrc-0-001519': '릿지당', 'mrc-0-002

### 8.파인튜닝 결과

In [52]:
"""
klue/bert-base - 기본 하이퍼파라미터에서 6기가 메모리, 3epoch 10분
***** eval metrics *****
  epoch        =     3.0
  eval_samples =     474
  exact_match  =    52.5
  f1           = 63.1449

klue/bert-base - 기본 하이퍼파라미터에서 batch16 10기가 메모리, 3epoch 10분
***** eval metrics *****
  epoch        =     3.0
  eval_samples =     474
  exact_match  =    52.5
  f1           = 63.7204


bespin-global/klue-bert-base-aihub-mrc - 기본 하이퍼파라미터에서 6기가 메모리, 3epoch 7분
 ***** eval metrics *****
  epoch        =     3.0
  eval_samples =     474
  exact_match  = 65.4167
  f1           = 73.7816

bespin-global/klue-bert-base-aihub-mrc - 기본 하이퍼파라미터에서 batch16 10기가 메모리, 3epoch 10분
***** eval metrics *****
  epoch        =     3.0
  eval_samples =     474
  exact_match  =   56.25
  f1           = 65.2439

yjgwak/klue-bert-base-finetuned-squad-kor-v1 -  기본 하이퍼파라미터에서 6기가 메모리, 3epoch 10분
***** eval metrics *****
  epoch        =     3.0
  eval_samples =     474
  exact_match  = 60.4167
  f1           = 67.9125

yjgwak/klue-bert-base-finetuned-squad-kor-v1 -  기본 하이퍼파라미터에서 batch16 10기가 메모리, 3epoch 10분
***** eval metrics *****
  epoch        =     3.0
  eval_samples =     474
  exact_match  = 60.8333
  f1           = 67.6382

Kdogs/klue-finetuned-squad_kor_v1 - bert-base, 기본 하이퍼파라미터에서 6기가 메모리, 3epoch 11분
***** eval metrics *****
  epoch        =     3.0
  eval_samples =     474
  exact_match  =   56.25
  f1           = 66.3594

Kdogs/klue-finetuned-squad_kor_v1 - bert-base, 기본 하이퍼파라미터에서 batch16 10기가 메모리, 3epoch 9분
***** eval metrics *****
  epoch        =     3.0
  eval_samples =     474
  exact_match  =   58.75
  f1           = 67.1615

timpal0l/mdeberta-v3-base-squad2 - 기본 하이퍼파라미터에서 11기가 메모리, 3epoch 22분
***** eval metrics *****
  epoch        =     3.0
  eval_samples =     652
  exact_match  = 54.5833
  f1           = 66.5511

timpal0l/mdeberta-v3-base-squad2 - 기본 하이퍼파라미터에서 batch16 17기가 메모리, 3epoch 18분
***** eval metrics *****
  epoch        =     3.0
  eval_samples =     652
  exact_match  = 56.6667
  f1           = 67.9844
"""

'\nklue/bert-base - 기본 하이퍼파라미터에서 6기가 메모리, 3epoch 10분\n***** eval metrics *****\n  epoch        =     3.0\n  eval_samples =     474\n  exact_match  =    52.5\n  f1           = 63.1449\n\nklue/bert-base - 기본 하이퍼파라미터에서 batch16 10기가 메모리, 3epoch 10분\n***** eval metrics *****\n  epoch        =     3.0\n  eval_samples =     474\n  exact_match  =    52.5\n  f1           = 63.7204\n\n\nbespin-global/klue-bert-base-aihub-mrc - 기본 하이퍼파라미터에서 6기가 메모리, 3epoch 7분\n ***** eval metrics *****\n  epoch        =     3.0\n  eval_samples =     474\n  exact_match  = 65.4167\n  f1           = 73.7816\n\nbespin-global/klue-bert-base-aihub-mrc - 기본 하이퍼파라미터에서 batch16 10기가 메모리, 3epoch 10분\n***** eval metrics *****\n  epoch        =     3.0\n  eval_samples =     474\n  exact_match  =   56.25\n  f1           = 65.2439\n\nyjgwak/klue-bert-base-finetuned-squad-kor-v1 -  기본 하이퍼파라미터에서 6기가 메모리, 3epoch 10분\n***** eval metrics *****\n  epoch        =     3.0\n  eval_samples =     474\n  exact_match  = 60.4167\n  f1       